In [1]:
# Hier werden die Bibs importiert

import pandas as pd
import numpy as np
from datetime import datetime
import sys
import matplotlib as mpl
import matplotlib.pyplot as plt
from statsmodels.tsa.seasonal import seasonal_decompose
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error, mean_absolute_percentage_error, r2_score
from sklearn.linear_model import LinearRegression
from statsmodels.tsa.holtwinters import ExponentialSmoothing
import pmdarima as pm
import itertools
import statsmodels
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from sklearn.preprocessing import OneHotEncoder
import os

# LSTM
# Preprocessing
from sklearn.preprocessing import MinMaxScaler
# Plotting
import seaborn as sns
import matplotlib.pyplot as plt

# deep learning
import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout

# Other
from IPython.display import Image
from time import time
import configparser
import subprocess
import warnings
import pprint
import os

# Prophet
from prophet import Prophet
from prophet.plot import add_changepoints_to_plot
from prophet.diagnostics import cross_validation
from prophet.diagnostics import performance_metrics
from prophet.plot import plot_cross_validation_metric
import holidays

# machine learning
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR



In [ ]:
# Hier werden die Anforderungen an den Anwender notiert
'''
csv-Datei; 2 Spalten; Datum, Name_Produkt
Start- und Enddatum der csv ergeben den betrachteten Zeitraum, 
Datumsformat der csv als dd.mm.yyyy
Aggloramation nur auf größere Zeiträume sinnvoll, keine Fehlerausgabe. Also von monatlichen Daten nicht auf Tagesdaten schließen
'''

In [109]:
# # # # Zwichenspeicher
# # # # "C:\Users\jlegt\Desktop\Master\Masterarbeit\Programmierung\Testdatensatz.csv"; D, -
# # # #  C:\Users\jlegt\Desktop\Master\Masterarbeit\Programmierung\Test2.csv; M, .
# # # # "C:\Users\jlegt\Desktop\Master\Masterarbeit\Programmierung\ind.csv"
# # # # Main Programm

# Eingaben Anwender
path, path_ind, decimal_separator, decimal_separator_ind, freq_timeseries, freq_forecast, agglo, fehler, selected_countries, horizont = eingaben()

# Import Zeitreihe
data, name_artikel = import_data(path, decimal_separator)

# Import Anreicherung
ind = import_ind(path_ind, decimal_separator_ind)

# Aggloramation 
cumulated_values = cumulate_values(data, freq_forecast)

# Datensatz in Trainings-, Validations- und Testdaten teilen
train_data, valid_data, test_data, train_valid_data = split_data(cumulated_values, horizont)

# Untersuchung Korrelation und merging Absatz und Indizes
result_df = korrelation (cumulated_values, ind, horizont)

# Datensatz mit Korrelationen in Trainings-, Validations- und Testdaten teilen
train_korr, valid_korr, test_korr, train_valid_korr = split_data(result_df, horizont)

# mehrere Diagramme der einzelnen Aggloramationen
create_plots_single(data, freq_timeseries, name_artikel)

# Diagramm der einzelnen Aggloramationen in einem Diagramm
create_plots_one(data, freq_timeseries, name_artikel)

# Zerlegung additiv
seasonal_decomposition_add(cumulated_values, name_artikel, agglo)

# Zerlegung multiplikativ
marker = seasonal_decomposition_mul(cumulated_values, name_artikel, agglo)

# ACF und PACF anzeigen
acf(cumulated_values['value'], lags=20) 

# lineare Regression mit Fehlern des Validierungsbereiches 
linReg_4, linRegValid_mae, linRegValid_mse, linRegValid_rmse, linRegValid_mape, linRegValid_r2 = linReg80(train_data, valid_data, name_artikel, agglo)

# lineare Regression mit Fehlern des Testbereiches
linReg_5, linRegTest_mae, linRegTest_mse, linRegTest_rmse, linRegTest_mape, linRegTest_r2 = linReg80(train_valid_data, test_data, name_artikel, agglo)

# Crosten mit Fehlern des Validierungsbereiches
Cros_4, CrosValid_mae, CrosValid_mse, CrosValid_mape, CrosValid_r2, CrosValid_rmse, best_alpha = croston80(train_data, valid_data, fehler, name_artikel, agglo)
print(best_alpha)
# Crosten mit Fehlern Prognosehorizont
Cros_5, CrosTest_mae, CrosTest_mse, CrosTest_mape, CrosTest_r2, CrosTest_rmse= croston100(train_valid_data, test_data, fehler, name_artikel, agglo, best_alpha)

# Holt-Winters add und mul mit Fehlern des Validierungsbereiches
if marker:
    (HWMul_4, HWAdd_4, HWMulValid_mae, HWMulValid_mse, HWMulValid_mape, HWMulValid_r2, HWMulValid_rmse, HWAddValid_mae, HWAddValid_mse, HWAddValid_mape, HWAddValid_r2, HWAddValid_rmse) = HoWi80(train_data, valid_data, agglo, fehler, name_artikel, marker, freq_forecast)
else:
    (HWAdd_4, HWAddValid_mae, HWAddValid_mse, HWAddValid_mape, HWAddValid_r2, HWAddValid_rmse) = HoWi80(train_data, valid_data, agglo, fehler, name_artikel, marker, freq_forecast)

# Holt-Winters add und mul mit Fehlern Prognosehorizont
if marker:
    (HWMul_5, HWAdd_5, HWMulTest_mae, HWMulTest_mse, HWMulTest_mape, HWMulTest_r2,HWMulTest_rmse, HWAddTest_mae, HWAddTest_mse, HWAddTest_mape, HWAddTest_r2, HWAddTest_rmse) = HoWi100(train_valid_data, test_data, agglo, fehler, name_artikel, marker, freq_forecast)
else:
    (HWAdd_5, HWAddTest_mae, HWAddTest_mse, HWAddTest_mape, HWAddTest_r2, HWAddTest_rmse) = HoWi100(train_valid_data, test_data, agglo, fehler, name_artikel, marker, freq_forecast)

# Arima univariat mit Fehlern im Testbereich
Arima_4, ArimaValid_mae, ArimaValid_mse, ArimaValid_mape, ArimaValid_r2, ArimaValid_rmse, order_ARIMA = ARIMA80(train_data, valid_data, fehler)

# Arima univariat mit Fehlern Prognosehorizont
Arima_5, ArimaTest_mae, ArimaTest_mse, ArimaTest_mape, ArimaTest_r2, ArimaTest_rmse = ARIMA100(train_valid_data, test_data, fehler, order_ARIMA)

# Sarima univariat mit Fehlern im Testbereich
Sarima_4, SarimaValid_mae, SarimaValid_mse, SarimaValid_mape, SarimaValid_r2, SarimaValid_rmse, order_SARIMA, seasonal_order_SARIMA = SARIMA80(train_data, valid_data, freq_forecast, fehler)

# Sarima univariat mit Fehlern Prognosehorizont
Sarima_5, SarimaTest_mae, SarimaTest_mse, SarimaTest_mape, SarimaTest_r2, SarimaTest_rmse = SARIMA100(train_valid_data, test_data, freq_forecast, fehler, order_SARIMA, seasonal_order_SARIMA)

# Prophet univariat ohne Feiertage mit Fehlern im Testbereich
FBPu_4, FBPuValid_mae, FBPuValid_mse, FBPuValid_mape, FBPuValid_r2, FBPuValid_rmse = prophet80uni(train_data, valid_data, freq_forecast)

# Prophet univariat ohne Feiertage mit Fehlern Prognosehorizont
FBPu_5, FBPuTest_mae, FBPuTest_mse, FBPuTest_mape, FBPuTest_r2, FBPuTest_rmse = prophet80uni(train_valid_data, test_data, freq_forecast)

# LSTM multivariat mit Fehlern im Testbereich
LSTM_4, LSTMValid_mae, LSTMValid_mse, LSTMValid_mape, LSTMValid_r2, LSTMValid_rmse = LSTM80(train_korr, valid_korr, fehler)

# LSTM multivariat mit Fehlern Prognosehorizont
LSTM_5, LSTMTest_mae, LSTMTest_mse, LSTMTest_mape, LSTMTest_r2, LSTMTest_rmse = LSTM80(train_valid_korr, test_korr, fehler)

# RandomForrest mit Fehlern im Testbereich
RF_4, RFValid_mae, RFValid_mse, RFValid_mape, RFValid_r2, RFValid_rmse = RF80(train_korr, valid_korr, fehler)

# RandomForrest mit Fehlern Prognosehorizont
RF_5, RFTest_mae, RFTest_mse, RFTest_mape, RFTest_r2, RFTest_rmse = RF80(train_valid_korr, test_korr, fehler)

# SVM mit Fehlern im Testbereich
SVM_4, SVMValid_mae, SVMValid_mse, SVMValid_mape, SVMValid_r2, SVMValid_rmse = SVM80(train_korr, valid_korr)

# SVM mit Fehlern Prognosehorizont
SVM_5, SVMTest_mae, SVMTest_mse, SVMTest_mape, SVMTest_r2, SVMTest_rmse = SVM80(train_valid_korr, test_korr)

# Prophet mulitvariat mit Fehlern Testbereich
FBPm_4, FBPmValid_mae, FBPmValid_mse, FBPmValid_mape, FBPmValid_r2, FBPmValid_rmse = prophet80multi(train_korr, valid_korr, selected_countries)

# Prophet mulitvariat mit Fehlern Prognosehorizont
FBPm_5, FBPmTest_mae, FBPmTest_mse, FBPmTest_mape, FBPmTest_r2, FBPmTest_rmse = prophet80multi(train_valid_korr, test_korr, selected_countries)

# Backward Feature Elimination mittels LSTM
(LSTM_b_4, LSTM_b_Valid_mae, LSTM_b_Valid_mse, LSTM_b_Valid_mape, 
LSTM_b_Valid_r2, LSTM_b_Valid_rmse, LSTM_b_model, LSTM_b_features)=backward_feature_elimination_valid(train_korr, valid_korr)

# Backward Feature Elimination mittels LSTM Ergebnisforecast
LSTM_b_5, LSTM_b_Test_mae, LSTM_b_Test_mse, LSTM_b_Test_mape, LSTM_b_Test_r2, LSTM_b_Test_rmse = backward_feature_forecast(train_valid_korr, test_korr, LSTM_b_model, LSTM_b_features)

# Fehlermetriken der Validierungsbereiche der Modelle vergleichen
ergebnis_valid(linRegValid_mae, linRegValid_mse, linRegValid_rmse, linRegValid_mape, linRegValid_r2,
                 CrosValid_mae, CrosValid_mse, CrosValid_rmse, CrosValid_mape, CrosValid_r2,
                 ArimaValid_mae, ArimaValid_mse, ArimaValid_rmse, ArimaValid_mape, ArimaValid_r2,
                 SarimaValid_mae, SarimaValid_mse, SarimaValid_rmse, SarimaValid_mape, SarimaValid_r2,
                 FBPuValid_mae, FBPuValid_mse, FBPuValid_rmse, FBPuValid_mape, FBPuValid_r2,
                 LSTMValid_mae, LSTMValid_mse, LSTMValid_rmse, LSTMValid_mape, LSTMValid_r2,
                 RFValid_mae, RFValid_mse, RFValid_rmse, RFValid_mape, RFValid_r2,
                 SVMValid_mae, SVMValid_mse, SVMValid_rmse, SVMValid_mape, SVMValid_r2,
                 FBPmValid_mae, FBPmValid_mse, FBPmValid_rmse, FBPmValid_mape, FBPmValid_r2,
                 HWAddValid_mae, HWAddValid_mse, HWAddValid_rmse, HWAddValid_mape, HWAddValid_r2,
                 HWMulValid_mae, HWMulValid_mse, HWMulValid_rmse, HWMulValid_mape, HWMulValid_r2,
                 LSTM_b_Valid_mae, LSTM_b_Valid_mse, LSTM_b_Valid_rmse, LSTM_b_Valid_mape, LSTM_b_Valid_r2)

# Fehlermetriken der Testbereiche der Modelle vergleichen
ergebnis_test(linRegTest_mae, linRegTest_mse, linRegTest_rmse, linRegTest_mape, linRegTest_r2,
                 CrosTest_mae, CrosTest_mse, CrosTest_rmse, CrosTest_mape, CrosTest_r2,
                 ArimaTest_mae, ArimaTest_mse, ArimaTest_rmse, ArimaTest_mape, ArimaTest_r2,
                 SarimaTest_mae, SarimaTest_mse, SarimaTest_rmse, SarimaTest_mape, SarimaTest_r2,
                 FBPuTest_mae, FBPuTest_mse, FBPuTest_rmse, FBPuTest_mape, FBPuTest_r2,
                 LSTMTest_mae, LSTMTest_mse, LSTMTest_rmse, LSTMTest_mape, LSTMTest_r2,
                 RFTest_mae, RFTest_mse, RFTest_rmse, RFTest_mape, RFTest_r2,
                 SVMTest_mae, SVMTest_mse, SVMTest_rmse, SVMTest_mape, SVMTest_r2,
                 FBPmTest_mae, FBPmTest_mse, FBPmTest_rmse, FBPmTest_mape, FBPmTest_r2,
                 HWAddTest_mae, HWAddTest_mse, HWAddTest_rmse, HWAddTest_mape, HWAddTest_r2,
                 HWMulTest_mae, HWMulTest_mse, HWMulTest_rmse, HWMulTest_mape, HWMulTest_r2,
                 LSTM_b_Test_mae, LSTM_b_Test_mse, LSTM_b_Test_rmse, LSTM_b_Test_mape, LSTM_b_Test_r2)



In [31]:
# Backward Feature Elimination mittels LSTM
# Funktion zum Erstellen des LSTM-Modells
def create_lstm_model(input_shape):
    model = Sequential(name='lstm_vale3')
    model.add(LSTM(units=input_shape[1], return_sequences=True, input_shape=input_shape))
    model.add(Dropout(0.2))
    model.add(LSTM(units=10, return_sequences=True))
    model.add(Dropout(0.2))
    model.add(LSTM(units=10, return_sequences=True))
    model.add(Dropout(0.2))
    model.add(LSTM(units=10))
    model.add(Dropout(0.2))
    model.add(Dense(units=1))
    return model

def backward_feature_elimination_valid(train_korr, valid_korr):
    train_copy = train_korr.copy()
    valid_copy = valid_korr.copy()
    train_copy.set_index('date', inplace=True)
    valid_copy.set_index('date', inplace=True)
    
    if 'year' in train_copy.columns and 'month' in train_copy.columns and 'day' in train_copy.columns:
        train_copy = train_copy.drop(['year', 'month', 'day'], axis=1)

    if 'year' in valid_copy.columns and 'month' in valid_copy.columns and 'day' in valid_copy.columns:
        valid_copy = valid_copy.drop(['year', 'month', 'day'], axis=1)

    df = pd.concat([train_copy, valid_copy], ignore_index=True)
    train_max = df.max()
    train_min = df.min()
    print(train_max)
    train = (df - train_min) / (train_max - train_min)

    size_train = len(train_copy)
    df_train = train.iloc[:size_train]
    df_valid = train.iloc[size_train:]

    y_valid = df_valid['value'].values.astype('float32')
    
    best_model = None
    best_features = None
    best_metric = float('inf')
    removed_features = []

    available_features = list(df_train.columns)
    available_features.remove('value')

    while len(available_features) > 6:
        current_best_metric = float('inf')
        current_best_feature = None

        for feature in available_features:
            current_features = available_features.copy()
            current_features.remove(feature)

            model = create_lstm_model((1, len(current_features)))
            model.compile(loss='mean_squared_error', optimizer='adam')

            X_train = df_train[current_features].values.reshape(-1, 1, len(current_features))
            y_train = df_train['value'].values.astype('float32')
            X_valid = df_valid[current_features].values.reshape(-1, 1, len(current_features))
            y_valid = df_valid['value'].values.astype('float32')

            model.fit(X_train, y_train, epochs=10, batch_size=30, shuffle=False, validation_data=(X_valid, y_valid), verbose=0)
            y_pred = model.predict(X_valid)

            metric_value = mean_absolute_error(y_valid, y_pred)

            if metric_value < current_best_metric:
                current_best_metric = metric_value
                current_best_feature = feature

        if current_best_metric < best_metric:
            best_metric = current_best_metric
            best_features = current_features.copy()
            available_features.remove(current_best_feature)
            removed_features.append(current_best_feature)
        else:
            break

    # Erstellen des endgültigen Modells mit den besten Merkmalen
    best_model = create_lstm_model((1, len(best_features)))
    best_model.compile(loss='mean_squared_error', optimizer='adam')

    X_train_final = df_train[best_features].values.reshape(-1, 1, len(best_features))
    y_train_final = df_train['value'].values.astype('float32')
    best_model.fit(X_train_final, y_train_final, epochs=100, batch_size=30, shuffle=False, verbose=0)

    # Plotten des Verlaufs der Prognose des endgültigen Modells
    y_pred_final = best_model.predict(X_train_final)

    y_pred_final = y_pred_final*(train_max['value']-train_min['value'])+train_min['value']
    y_train_final = y_train_final*(train_max['value']-train_min['value'])+train_min['value']

    plt.figure(figsize=(8, 6))
#     plt.plot(range(1, len(y_train_final) + 1), y_train_final, label='Trainingsdaten')
#     plt.plot(range(2, len(y_train_final) + 1), y_pred_final[:-1], color='red', label='Vorhersagen (Train)')
#     plt.title('LSTM Vorhersagen vs. Trainingsdaten')
#     plt.xlabel('Datum')
#     plt.ylabel('Wert')
#     plt.legend()
#     plt.show()

    # Kombination der besten 6 Features ausgeben
    print("Beste Merkmalskombination:", best_features)

    # Entfernte Merkmale ausgeben
    print("Entfernte Merkmale:", removed_features)

    # Gesamtes Modell ausgeben
    print("Gesamtes Modell:")
    best_model.summary()

    # Plot für den Forecast mit den Validierungsdaten
    X_valid_final = df_valid[best_features].values.reshape(-1, 1, len(best_features))
    y_pred_valid = best_model.predict(X_valid_final)

    y_pred_valid = y_pred_valid*(train_max['value'] -train_min['value'])+train_min['value']
    y_valid = y_valid*(train_max['value']-train_min['value'])+train_min['value']

    # Plot für Trainingsdaten und Vorhersagen (Train)
    plt.plot(range(1, len(y_train_final) + 1), y_train_final, color='blue', label='Trainingsdaten')
    plt.plot(range(len(y_train_final), len(y_train_final) + len(y_valid)), y_valid, color='green', label='Validierungsdaten')
    plt.plot(range(len(y_train_final), len(y_train_final) + len(y_pred_valid)), y_pred_valid, color='red', linestyle='--', label='Vorhersagen (Validierung)')
    plt.plot(range(2, len(y_train_final) + 1), y_pred_final[:-1], color='orange', label='Vorhersagen (Train)')
#     plt.title('LSTM Vorhersagen vs. Trainings- und Validierungsdaten')
    plt.xlabel('Datum')
    plt.ylabel('Wert')
    plt.legend()
    print(y_pred_valid)
    # Speichern als PNG
    plt.savefig('LSTM_BFS_train.png')
    plt.tight_layout()


    
    LSTM_b_Valid_mae = mean_absolute_error(y_valid, y_pred_valid)
    LSTM_b_Valid_mse = mean_squared_error(y_valid, y_pred_valid)
    LSTM_b_Valid_mape = mean_absolute_percentage_error(y_valid, y_pred_valid)
    LSTM_b_Valid_r2 = r2_score(y_valid, y_pred_valid)
    LSTM_b_Valid_rmse = np.sqrt(mean_squared_error(y_valid, y_pred_valid))
                                
    LSTM_b_model = best_model
    LSTM_b_features = best_features
    
    return y_pred_valid, LSTM_b_Valid_mae, LSTM_b_Valid_mse, LSTM_b_Valid_mape, LSTM_b_Valid_r2, LSTM_b_Valid_rmse, LSTM_b_model, LSTM_b_features


In [32]:
# Backward Feature Elimination mittels LSTM
# Funktion zum Erstellen des LSTM-Modells
def backward_feature_forecast(train_valid_korr, test_korr, LSTM_b_model, LSTM_b_features):
    train_copy = train_valid_korr.copy()
    valid_copy = test_korr.copy()
    train_copy.set_index('date', inplace=True)
    valid_copy.set_index('date', inplace=True)
    
    

    df = pd.concat([train_copy, valid_copy], ignore_index=True)
    train_max = df.max()
    train_min = df.min()
#     print(train_max)
    train = (df - train_min) / (train_max - train_min)

    size_train = len(train_copy)
    df_train = train.iloc[:size_train]
    df_valid = train.iloc[size_train:]

    y_valid = df_valid['value'].values.astype('float32')

    best_features = LSTM_b_features
    best_model = LSTM_b_model
    
    X_train_final = df_train[best_features].values.reshape(-1, 1, len(best_features))
    y_train_final = df_train['value'].values.astype('float32')
    best_model.fit(X_train_final, y_train_final, epochs=100, batch_size=30, shuffle=False, verbose=0)

    # Plotten des Verlaufs der Prognose des endgültigen Modells
    y_pred_final = best_model.predict(X_train_final)

    y_pred_final = y_pred_final*(train_max['value']-train_min['value'])+train_min['value']
    y_train_final = y_train_final*(train_max['value']-train_min['value'])+train_min['value']

#     plt.figure(figsize=(8, 6))
#     plt.plot(range(1, len(y_train_final) + 1), y_train_final, label='Trainingsdaten')
#     plt.plot(range(2, len(y_train_final) + 1), y_pred_final[:-1], color='red', label='Vorhersagen (Train)')
#     plt.title('LSTM Vorhersagen vs. Trainingsdaten')
#     plt.xlabel('Datum')
#     plt.ylabel('Wert')
#     plt.legend()
#     plt.show()


    # Plot für den Forecast mit den Validierungsdaten
    X_valid_final = df_valid[best_features].values.reshape(-1, 1, len(best_features))
    y_pred_valid = best_model.predict(X_valid_final)

    y_pred_valid = y_pred_valid*(train_max['value'] -train_min['value'])+train_min['value']
    y_valid = y_valid*(train_max['value']-train_min['value'])+train_min['value']
    
    plt.figure(figsize=(8, 6))
    # Plot für Trainingsdaten und Vorhersagen (Train)
    plt.plot(range(1, len(y_train_final) + 1), y_train_final, color='blue', label='Trainingsdaten')
    plt.plot(range(len(y_train_final), len(y_train_final) + len(y_valid)), y_valid, color='green', label='Validierungsdaten')
    plt.plot(range(len(y_train_final), len(y_train_final) + len(y_pred_valid)), y_pred_valid, color='red', linestyle='--', label='Vorhersagen (Validierung)')
    plt.plot(range(2, len(y_train_final) + 1), y_pred_final[:-1], color='orange', label='Vorhersagen (Train)')
#     plt.title('LSTM Vorhersagen vs. Trainings- und Validierungsdaten')
    plt.xlabel('Datum')
    plt.ylabel('Wert')
    plt.legend()
    
    
    # Speichern als PNG
    plt.savefig('LSTM_BFS_test.png')
    plt.tight_layout()
    print(y_pred_valid)
    
    LSTM_b_Test_mae = mean_absolute_error(y_valid, y_pred_valid)
    LSTM_b_Test_mse = mean_squared_error(y_valid, y_pred_valid)
    LSTM_b_Test_mape = mean_absolute_percentage_error(y_valid, y_pred_valid)
    LSTM_b_Test_r2 = r2_score(y_valid, y_pred_valid)
    LSTM_b_Test_rmse = np.sqrt(mean_squared_error(y_valid, y_pred_valid))
                                

    
    return y_pred_valid, LSTM_b_Test_mae, LSTM_b_Test_mse, LSTM_b_Test_mape, LSTM_b_Test_r2, LSTM_b_Test_rmse


In [33]:
def eingaben():
    # Eingaben Anwender; in eigenes Unterprogramm

    # Pfad zur csv_Zeitreihe
    path = str(input("Bitte geben Sie den Pfad zur CSV-Datei der Zeitreihe ein: "))
    # Pfad zur csv_Indizes
    path_ind = str(input("Bitte geben Sie den Pfad zur CSV-Datei der Anreicherung ein: "))
    # Eingabe Dezimalzeichen Menge
    decimal_separator = str(input("Bitte geben Sie das Dezimaltrennzeichen der CSV-Datei Zeitreihe ein (z.B.: '.' oder ','). Falls unbekannt, CSV-Datei via Text-Editor öffnen. Falls keins enthalten (bspw. Stückzahl) drücken Sie die Enter-Taste  "))
    # Eingabe Dezimalzeichen Menge
    decimal_separator_ind = str(input("Bitte geben Sie das Dezimaltrennzeichen der CSV-Datei Anreicherung ein (z.B.: '.' oder ','). Falls unbekannt, CSV-Datei via Text-Editor öffnen. Falls keins enthalten (bspw. Stückzahl) drücken Sie die Enter-Taste  "))
    # Eingabe Aggloramation Zeiten
    freq_timeseries = str(input("Bitte geben Sie die Aggloramation ein, in dem Ihre Daten angegeben sind (D, W, M, Q): ").upper())
    # Welche Betrachtungsaggloramation
    freq_forecast = str(input("Bitte geben Sie die Aggloramation ein, in dem Sie den Forecast betrachten wollen (D, W, M, Q): ").upper())
    # Iteration nach welchem Fehler
    fehler = str(input("Nach welchem Fehler soll iteriert werden (MAE, MSE, MAPE, R2, RMSE)? ").upper())
    # Eingabeaufforderung für Länderkürzel Feiertage bei Prophet
    selected_countries = input("Geben Sie die Länderkürzel durch Leerzeichen getrennt ein (z.B. DE US FRA): ").upper().split()
    # Prognosehorizont festlegen
    horizont = int(input("Wie viele Perioden soll der Forecast berechnen? "))
    
    # Zuschneiden Importlink
    path = path.strip('"')
    path_ind = path_ind.strip('"')

    # Überprüfung, ob die Eingabe leer ist
    if not decimal_separator:  
        decimal_separator = '.' 
    # Überprüfung, ob die Eingabe leer ist
    if not decimal_separator_ind:  
        decimal_separator_ind = '.' 
        
    # Kontrolle Aggloramation Zeitreihe
    valid_freq = ['D', 'W', 'M', 'Q']

    if freq_timeseries in valid_freq:
        print("Das eingegebene Zeitintervall ist:", freq_timeseries) # kann raus
    else:
        print("Ungültiges Zeitintervall. Bitte geben Sie D, W, M oder Q ein.")
        sys.exit()

    # Kontrolle Aggloramation Forecast
    valid_freq = ['D', 'W', 'M', 'Q']

    if freq_forecast in valid_freq:
        print("Das eingegebene Zeitintervall ist:", freq_forecast) # kann raus

        # Festlegung der Variable agglo basierend auf freq_forecast
        if freq_forecast == 'D':
            agglo = 'täglich'
        elif freq_forecast == 'W':
            agglo = 'wöchentlich'
        elif freq_forecast == 'M':
            agglo = 'monatlich'
        elif freq_forecast == 'Q':
            agglo = 'quartalsweise'
    else:
        print("Ungültiges Zeitintervall. Bitte geben Sie D, W, M oder Q ein.")
        sys.exit()
    return path, path_ind, decimal_separator, decimal_separator_ind, freq_timeseries, freq_forecast, agglo, fehler, selected_countries, horizont

In [34]:
# Import der Zeitreihe
def import_data(path, decimal_separator):
    possible_delimiters = [';', ':', '|', '\t', ',']  # Liste der möglichen Trennzeichen
    
    for delimiter in possible_delimiters:
        try:
            data = pd.read_csv(path, delimiter=delimiter, decimal=decimal_separator)
            # Wenn keine Fehler auftritt, breche die Schleife ab und gib die Daten zurück
            break
        except pd.errors.ParserError:
            # Wenn ein Fehler auftritt, versuche das nächste Trennzeichen
            continue
    else:
        # Falls kein passendes Trennzeichen gefunden wurde
        print("Kein gültiges Trennzeichen gefunden.")
        return None

    
    data[data.columns[0]] = pd.to_datetime(data[data.columns[0]], format='%d.%m.%Y')
    data = data.sort_values(data.columns[0])
    
    # Abfrage des Artikelnamens
    if data is not None and len(data.columns) > 1:
        name_artikel = data.columns[1]
    else:
        name_artikel = 'Artikel'
    
    # Umbenennung der Spalten
    data = data.rename(columns={data.columns[0]: 'date', data.columns[1]: 'value'})
    # Finde den Index des ersten Eintrags in der Spalte 'value', der nicht Null ist
    first_nonzero_index = data['value'].ne(0).idxmax()

    # Schneide den DataFrame ab dem ersten Eintrag, der nicht Null ist, bis zum Ende ab
    data = data.loc[first_nonzero_index:]

    return data, name_artikel

In [35]:
# Import Anreicherung

def import_ind(path, decimal_separator):
    possible_delimiters = [';', ':', '|', '\t', ',']  # Liste der möglichen Trennzeichen
    
    for delimiter in possible_delimiters:
        try:
            ind = pd.read_csv(path, delimiter=delimiter, decimal=decimal_separator)
            # Wenn keine Fehler auftritt, breche die Schleife ab und gib die Daten zurück
            break
        except pd.errors.ParserError:
            # Wenn ein Fehler auftritt, versuche das nächste Trennzeichen
            continue
    else:
        # Falls kein passendes Trennzeichen gefunden wurde
        print("Kein gültiges Trennzeichen gefunden.")
        return None
    
    ind[ind.columns[0]] = pd.to_datetime(ind[ind.columns[0]], format='%d.%m.%Y')
    ind = ind.sort_values(ind.columns[0])
    
    # Umbenennung der Spalten
    ind = ind.rename(columns={ind.columns[0]: 'date_ind'})
    
    return (ind)



In [36]:
# Aggloramation der zu betrachtenden Periodengröße

def cumulate_values(data, freq_forecast):
    # Start- und Enddatum der Zeitreihe identifizieren
    start_date = data['date'].min()
    end_date = data['date'].max()
    
    # Erstellen einer leeren Tabelle mit den Tagen von start_date bis end_date
    dates = pd.date_range(start_date, end_date, freq='D')
    days = pd.DataFrame({'date': dates, 'value': 0})
    
    # Hinzufügen der Anzahl für jeden Tag des aktuellen Artikels
    for date in dates:
        datum = date.date()
        anzahl_sum = data.loc[data['date'].dt.date == pd.Timestamp(date).date(), 'value'].sum()
        if anzahl_sum > 0:
            days.loc[days['date'] == date, 'value'] = anzahl_sum
            
    if freq_forecast == 'D':
        cumulated_values = days
    elif freq_forecast == 'W':
        cumulated_values = days.groupby(pd.Grouper(key='date', freq='W-MON')).sum().reset_index()
    elif freq_forecast == 'M':
        cumulated_values = days.groupby(pd.Grouper(key='date', freq='MS')).sum().reset_index()
    elif freq_forecast == 'Q':
        cumulated_values = days.groupby(pd.Grouper(key='date', freq='QS')).sum().reset_index()
    
    return cumulated_values


In [38]:
# Datensatz in Trainings- und Testdaten teilen

def split_data(data, horizont):
    # Abtrennen der letzten 'horizont' Spalten als test_data
    test_data = data.iloc[-horizont:]
    
    # Aufteilung der verbleibenden Daten in train_data und valid_data
    train_valid_data = data.iloc[:-horizont]
    train_size = int(len(train_valid_data) * 0.8)  # 80% der Daten für das Training
    train_data = train_valid_data.iloc[:train_size]
    valid_data = train_valid_data.iloc[train_size:]
    
    return train_data, valid_data, test_data, train_valid_data


In [39]:
# Untersuchung möglicher Korrelationen und anschließender Aneinanderfügung

def korrelation (cumulated_values, ind, horizont):

#     merged_df = pd.merge(cumulated_values, ind, left_on='date', right_on='date_ind', how='left')
    merged_df = pd.merge(cumulated_values, ind, left_on='date', right_on='date_ind', how='outer')
    # Sortieren nach Datum
    merged_df = merged_df.sort_values('date_ind')
    # Löschen der Spalte 'date_ind'
    merged_df.drop(columns=['date_ind'], inplace=True)

#     print(merged_df)
    
    # Separieren von verk (zweite Spalte) und anr (Spalten danach)
    verk = merged_df.iloc[:, 1:2]  # die zweite Spalte
    anr = merged_df.iloc[:, 1:]    # alle Spalten ab der dritten Spalte

    # Anzahl der Lags, die erstellt werden sollen
    num_lags = 24

    # Erstellen des DataFrames für die Lag-Daten
    lagged_anr = pd.DataFrame(index=range(horizont, len(anr)))  # Starten Sie bei 'horizont'

    # Liste für den Output
    output_list = []

    # Berechnen der Korrelation zwischen der ersten Spalte von "verk" und den Lags für jede Spalte von "anr"
    for column in anr.columns:
        # DataFrame für die Lags der aktuellen Spalte erstellen, beginnend ab 'horizont'
        lagged_column = pd.concat([anr[column].shift(i) for i in range(horizont, num_lags + 1)], axis=1)
        lagged_column.columns = [f"{column}_lag{i}" for i in range(horizont, num_lags + 1)]

        # Berechnen der Korrelation zwischen der ersten Spalte von "verk" und den Lags der aktuellen Spalte
        correlations = {}
        for lag_column in lagged_column.columns:
            correlation = verk.iloc[:, 0].corr(lagged_column[lag_column].dropna())
            correlations[lag_column] = correlation

        # Den Lag mit der höchsten Korrelation für die aktuelle Spalte von "anr" finden
        best_lag = max(correlations, key=correlations.get)
        best_correlation = correlations[best_lag]

        # Füge den Output zur Liste hinzu
        output_list.append((column, best_lag, best_correlation))

    # Sortieren der Liste nach den Korrelationen mit dem größten Betrag
    sorted_output_list = sorted(output_list, key=lambda x: abs(x[2]), reverse=True)

    # Behalte nur die ersten 15 Einträge der sortierten Liste
    top_5_output_list = sorted_output_list[:15]

    # Output anzeigen
    print("Beste Lags für erste Spalte von 'verk' mit der entsprechenden Korrelation:")
    for item in top_5_output_list:
        column, best_lag, correlation = item
        print(f"{column}: {best_lag} (Korrelation: {correlation})")

    # DataFrame für die Lags erstellen
    lagged_df = pd.DataFrame()

    # Durch top_5_output_list iterieren
    for column_name, lag_string, _ in top_5_output_list:
        # Den String zwischen den runden Klammern extrahieren
        extracted_string = lag_string.split('(')[-1].split(')')[0]

        # Den Namen der Originalspalte identifizieren
        original_column_name = extracted_string.split('_lag')[0]

        # Das Lag-Level extrahieren
        lag_level = int(extracted_string.split('_lag')[-1])

        # Das Lag berechnen
        lagged_column = anr[original_column_name].shift(lag_level)

        # Den Namen des generierten Lags erstellen
        lagged_column_name = f"{original_column_name}_lag{lag_level}"

        # Den generierten Lag dem DataFrame hinzufügen
        lagged_df[lagged_column_name] = lagged_column

    # Output anzeigen
#     print(lagged_df)

    result_df = pd.merge(verk, lagged_df, left_index=True, right_index=True)
#     print(result_df)
    # Hinzufügen der Spalte 'date' von merged_df zu results_df
    result_df['date'] = merged_df['date']

    # Spalte 'date' entfernen
    date_column = result_df.pop('date')

    # Spalte an die erste Position einfügen
    result_df.insert(0, 'date', date_column)

    # Entfernen der Zeilen mit NaN-Werten
    result_df = result_df.dropna()

    
    # Extrahieren von Jahr, Monat und Tag und Erstellen der neuen Spalten
    result_df['year'] = result_df['date'].dt.year
    result_df['month'] = result_df['date'].dt.month
    result_df['day'] = result_df['date'].dt.day
    
    print(result_df)

    return result_df


In [84]:
# mehrere Diagramme der Einzelnen Aggloramationen

def create_plots_single(cumulated_values, freq_timeseries, name_artikel):
    plt.figure(figsize=(8, 6))
    titles = {'D': 'Täglich', 'W': 'Wöchentlich', 'M': 'Monatlich', 'Q': 'Quartalsweise'}
    
    if freq_timeseries == 'D':
        plt.subplot(2, 2, 1)
        plt.plot(cumulated_values['date'], cumulated_values['value'], label='Täglich')
        plt.xlabel('Datum')
        plt.ylabel('Anzahl')
#         plt.title('Tägliche Anzahl ' + name_artikel)
        plt.xticks(rotation=45)
        plt.legend()

        plt.subplot(2, 2, 2)
        plt.plot(cumulated_values.groupby(pd.Grouper(key='date', freq='W-MON')).sum().reset_index()['date'],
                 cumulated_values.groupby(pd.Grouper(key='date', freq='W-MON')).sum().reset_index()['value'], 
                 label='Wöchentlich')
        plt.xlabel('Datum')
        plt.ylabel('Anzahl')
#         plt.title('Wöchentliche Anzahl ' + name_artikel)
        plt.xticks(rotation=45)
        plt.legend()

        plt.subplot(2, 2, 3)
        plt.plot(cumulated_values.groupby(pd.Grouper(key='date', freq='MS')).sum().reset_index()['date'],
                 cumulated_values.groupby(pd.Grouper(key='date', freq='MS')).sum().reset_index()['value'], 
                 label='Monatlich')
        plt.xlabel('Datum')
        plt.ylabel('Anzahl')
#         plt.title('Monatliche Anzahl ' + name_artikel)
        plt.xticks(rotation=45)
        plt.legend()

        plt.subplot(2, 2, 4)
        plt.plot(cumulated_values.groupby(pd.Grouper(key='date', freq='QS')).sum().reset_index()['date'],
                 cumulated_values.groupby(pd.Grouper(key='date', freq='QS')).sum().reset_index()['value'], 
                 label='Quartalsweise')
        plt.xlabel('Datum')
        plt.ylabel('Anzahl')
#         plt.title('Quartalsweise Anzahl ' + name_artikel)
        plt.xticks(rotation=45)
        plt.legend()

    elif freq_timeseries == 'W':
        plt.subplot(2, 2, 1)
        plt.plot(cumulated_values.groupby(pd.Grouper(key='date', freq='W-MON')).sum().reset_index()['date'],
                 cumulated_values.groupby(pd.Grouper(key='date', freq='W-MON')).sum().reset_index()['value'], 
                 label='Wöchentlich')
        plt.xlabel('Datum')
        plt.ylabel('Anzahl')
#         plt.title('Wöchentliche Anzahl ' + name_artikel)
        plt.xticks(rotation=45)
        plt.legend()

        plt.subplot(2, 2, 2)
        plt.plot(cumulated_values.groupby(pd.Grouper(key='date', freq='MS')).sum().reset_index()['date'],
                 cumulated_values.groupby(pd.Grouper(key='date', freq='MS')).sum().reset_index()['value'], 
                 label='Monatlich')
        plt.xlabel('Datum')
        plt.ylabel('Anzahl')
#         plt.title('Monatliche Anzahl ' + name_artikel)
        plt.xticks(rotation=45)
        plt.legend()

        plt.subplot(2, 2, 3)
        plt.plot(cumulated_values.groupby(pd.Grouper(key='date', freq='QS')).sum().reset_index()['date'],
                 cumulated_values.groupby(pd.Grouper(key='date', freq='QS')).sum().reset_index()['value'], 
                 label='Quartalsweise')
        plt.xlabel('Datum')
        plt.ylabel('Anzahl')
#         plt.title('Quartalsweise Anzahl ' + name_artikel)
        plt.xticks(rotation=45)
        plt.legend()

    elif freq_timeseries == 'M':
        plt.subplot(2, 1, 1)
        plt.plot(cumulated_values.groupby(pd.Grouper(key='date', freq='MS')).sum().reset_index()['date'],
                 cumulated_values.groupby(pd.Grouper(key='date', freq='MS')).sum().reset_index()['value'], 
                 label='Monatlich')
        plt.xlabel('Datum')
        plt.ylabel('Anzahl')
#         plt.title('Monatliche Anzahl ' + name_artikel)
        plt.xticks(rotation=45)
        plt.legend()

        plt.subplot(2, 1, 2)
        plt.plot(cumulated_values.groupby(pd.Grouper(key='date', freq='QS')).sum().reset_index()['date'],
                 cumulated_values.groupby(pd.Grouper(key='date', freq='QS')).sum().reset_index()['value'], 
                 label='Quartalsweise')
        plt.xlabel('Datum')
        plt.ylabel('Anzahl')
#         plt.title('Quartalsweise Anzahl ' + name_artikel)
        plt.xticks(rotation=45)
        plt.legend()

    elif freq_timeseries == 'Q':
        plt.plot(cumulated_values.groupby(pd.Grouper(key='date', freq='QS')).sum().reset_index()['date'],
                 cumulated_values.groupby(pd.Grouper(key='date', freq='QS')).sum().reset_index()['value'], 
                 label='Quartalsweise')
        plt.xlabel('Datum')
        plt.ylabel('Absatz [Stück]')
#         plt.title('Quartalsweise Anzahl ' + name_artikel)
        plt.xticks(rotation=45)
        plt.legend()

    plt.tight_layout()
    plt.savefig('Verläufe einzeln.png')
    plt.show()


In [85]:
# Diagramm der einzelnen Aggloramationen in einem Diagramm

def create_plots_one(cumulated_values, freq_timeseries, name_artikel):
    plt.figure(figsize=(8, 6))
    titles = {'D': 'Täglich', 'W': 'Wöchentlich', 'M': 'Monatlich', 'Q': 'Quartalsweise'}
    
    if freq_timeseries == 'D':
        plt.plot(cumulated_values['date'], cumulated_values['value'], label='Täglich')
        plt.plot(cumulated_values.groupby(pd.Grouper(key='date', freq='W-MON')).sum().reset_index()['date'],
                 cumulated_values.groupby(pd.Grouper(key='date', freq='W-MON')).sum().reset_index()['value'], 
                 label='Wöchentlich', linestyle='--')
        plt.plot(cumulated_values.groupby(pd.Grouper(key='date', freq='MS')).sum().reset_index()['date'],
                 cumulated_values.groupby(pd.Grouper(key='date', freq='MS')).sum().reset_index()['value'], 
                 label='Monatlich', linestyle='-.')
        plt.plot(cumulated_values.groupby(pd.Grouper(key='date', freq='QS')).sum().reset_index()['date'],
                 cumulated_values.groupby(pd.Grouper(key='date', freq='QS')).sum().reset_index()['value'], 
                 label='Quartalsweise', linestyle=':')

    elif freq_timeseries == 'W':
        plt.plot(cumulated_values.groupby(pd.Grouper(key='date', freq='W-MON')).sum().reset_index()['date'],
                 cumulated_values.groupby(pd.Grouper(key='date', freq='W-MON')).sum().reset_index()['value'], 
                 label='Wöchentlich', linestyle='--')
        plt.plot(cumulated_values.groupby(pd.Grouper(key='date', freq='MS')).sum().reset_index()['date'],
                 cumulated_values.groupby(pd.Grouper(key='date', freq='MS')).sum().reset_index()['value'], 
                 label='Monatlich', linestyle='-.')
        plt.plot(cumulated_values.groupby(pd.Grouper(key='date', freq='QS')).sum().reset_index()['date'],
                 cumulated_values.groupby(pd.Grouper(key='date', freq='QS')).sum().reset_index()['value'], 
                 label='Quartalsweise', linestyle=':')

    elif freq_timeseries == 'M':
        plt.plot(cumulated_values.groupby(pd.Grouper(key='date', freq='MS')).sum().reset_index()['date'],
                 cumulated_values.groupby(pd.Grouper(key='date', freq='MS')).sum().reset_index()['value'], 
                 label='Monatlich', linestyle='-.')
        plt.plot(cumulated_values.groupby(pd.Grouper(key='date', freq='QS')).sum().reset_index()['date'],
                 cumulated_values.groupby(pd.Grouper(key='date', freq='QS')).sum().reset_index()['value'], 
                 label='Quartalsweise', linestyle=':')

    elif freq_timeseries == 'Q':
        plt.plot(cumulated_values.groupby(pd.Grouper(key='date', freq='QS')).sum().reset_index()['date'],
                 cumulated_values.groupby(pd.Grouper(key='date', freq='QS')).sum().reset_index()['value'], 
                 label='Quartalsweise')

    plt.xlabel('Datum')
    plt.ylabel('Absatz [Stück]')
#     plt.title('Anzahl ' + name_artikel)
    plt.xticks(rotation=45)
    plt.legend()
    plt.tight_layout()
    plt.savefig('Verläufe zusammen.png')
    plt.show()


In [42]:
# Zeitreihenzerlegung mit dem multiplikativen Modell

def seasonal_decomposition_mul(data, name_artikel, agglo):
    # DataFrame mit DateTime-Index erstellen, ohne den ursprünglichen DataFrame zu ändern
    data_copy = data.copy()
    data_copy.index = pd.to_datetime(data_copy['date'])
    
    # Marker für Anwesenheit '0' values
    marker = False
    # Prüfen, ob der DataFrame keine Anzahl "0" enthält
    if (data_copy["value"] != 0).all():
        print('Zerlegung multiplikativ')
        # Marker für '0' values
        marker = True
        # Zerlegung der Zeitreihe mit model='mul'
        decompose_result = seasonal_decompose(data_copy["value"], model='mul')
        
        # Zerlegungsplots anzeigen und Titel setzen
        fig, (ax1, ax2, ax3, ax4) = plt.subplots(4, 1, figsize=(8, 6))
        decompose_result.observed.plot(ax=ax1)
        ax1.set_title("Observed")
        decompose_result.trend.plot(ax=ax2)
        ax2.set_title("Trend")
        decompose_result.seasonal.plot(ax=ax3)
        ax3.set_title("Seasonal")
        decompose_result.resid.plot(ax=ax4)
        ax4.set_title("Residual")
#         plt.suptitle("Zerlegung multiplikativ für " + name_artikel + ' für ' + agglo)
        plt.tight_layout(rect=[0, 0.03, 1, 0.95])
        plt.savefig('Zerlegung mul.png')
        plt.show()
    
    else:
        print("Der DataFrame enthält Anzahl '0', daher wird keine Zerlegung mit model='mul' durchgeführt.")
    
    return marker

In [43]:
# Zeitreihenzerlegung mit dem additiven Modell


def seasonal_decomposition_add(data, name_artikel, agglo):
    # DataFrame mit DateTime-Index erstellen, ohne den ursprünglichen DataFrame zu ändern
    data_copy = data.copy()
    data_copy.index = pd.to_datetime(data_copy['date'])
    print('Zerlegung additiv')
    # Zerlegung der Zeitreihe mit model='add'
    decompose_result = seasonal_decompose(data_copy["value"], model='add')

    # Zerlegungsplots anzeigen und Titel setzen
    fig, (ax1, ax2, ax3, ax4) = plt.subplots(4, 1, figsize=(8, 6))
    decompose_result.observed.plot(ax=ax1)
    ax1.set_title("Observed")
    decompose_result.trend.plot(ax=ax2)
    ax2.set_title("Trend")
    decompose_result.seasonal.plot(ax=ax3)
    ax3.set_title("Seasonal")
    decompose_result.resid.plot(ax=ax4)
    ax4.set_title("Residual")
#     plt.suptitle("Zerlegung additiv für " + name_artikel + ' für ' + agglo)
    plt.tight_layout(rect=[0, 0.03, 1, 0.95])
    plt.savefig('Zerlegung add.png')
    plt.show()


In [92]:
# ACF + PACF bessere Abbildung

import matplotlib.pyplot as plt
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf

def acf(cumulated_values, lags):
    # Plot der ACF und PACF
    fig, axes = plt.subplots(1, 2, figsize=(8, 6))
    
    # Berechnung und Plot der ACF
    plot_acf(cumulated_values, lags, ax=axes[0])
    axes[0].set_title('a)', fontsize=12, loc='left')
    axes[0].set_ylabel('Korrelationswert', fontsize=12)
    axes[0].set_xlabel('Verzögerung [Periode]', fontsize=12)
    
    # Berechnung und Plot der PACF
    plot_pacf(cumulated_values, lags, ax=axes[1])
    axes[1].set_title('b)', fontsize=12, loc='left')
    axes[1].set_ylabel('Korrelationswert', fontsize=12)
    axes[1].set_xlabel('Verzögerung [Periode]', fontsize=12)
    
    plt.tight_layout()
    plt.savefig('ACF+PACF.png')
    plt.show()


In [45]:
def linReg80(train_data, valid_data, name_artikel, agglo):
    # def linReg80(train_data, valid_data, test_data, name_artikel, agglo):
    train_dates = train_data['date']
    valid_dates = valid_data['date']


    # Extrahieren Sie die Features und Zielvariablen aus den Daten
    Y_train = train_data["value"].values
    Y_valid = valid_data["value"].values

    # Extrahieren des Index als Features
    X_train = train_data.index.values.reshape(-1, 1)
    X_valid = valid_data.index.values.reshape(-1, 1)

    # Lineare Regression für Trainingsdaten durchführen
    lin_reg = LinearRegression()
    lin_reg.fit(X_train, Y_train)

    # Prognose für Validierungsdaten durchführen
    y_valid_pred = lin_reg.predict(X_valid)

    # Berechnung der Fehlerindikatoren für Validierungsdaten
    linRegValid_mae = mean_absolute_error(Y_valid, y_valid_pred)
    linRegValid_mse = mean_squared_error(Y_valid, y_valid_pred)
    linRegValid_rmse = np.sqrt(linRegValid_mse)
    linRegValid_mape = mean_absolute_percentage_error(Y_valid, y_valid_pred)
    linRegValid_r2 = r2_score(Y_valid, y_valid_pred)

    
    # Berechnung der Fehlerindikatoren für Testdaten
#     linRegTest_mae = mean_absolute_error(Y_test, y_test_pred)
#     linRegTest_mse = mean_squared_error(Y_test, y_test_pred)
#     linRegTest_rmse = np.sqrt(linRegTest_mse)
#     linRegTest_mape = mean_absolute_percentage_error(Y_test, y_test_pred)
#     linRegTest_r2 = r2_score(Y_test, y_test_pred)

   # Setze die Parameter gemäß den Vorgaben
    fontsize = 12
    fontsize_legende = 8
    figsize = (8, 6)

    # Daten und Regression plotten
    plt.figure(figsize=figsize)

    # Plot für Trainings-, Validierungs- und Testdaten
    plt.plot(train_dates, Y_train, label='Trainingsdaten', color='blue')  # Hellblau für Trainingsdaten
    plt.plot(valid_dates, Y_valid, label='Validierungsdaten (wahre Werte)', color='green')  # Orange für Validierungsdaten
    plt.plot(valid_dates, y_valid_pred, linestyle='--', label='Validierungsprognose', color='red')  # Rot für Validierungsprognose
    plt.xlabel('Datum', fontsize=fontsize)
    plt.ylabel('Anzahl', fontsize=fontsize)
    plt.legend(fontsize=fontsize_legende)
#     plt.title("Lineare Regression und Prognose für " + name_artikel + " für " + agglo, fontsize=fontsize)

    plt.tight_layout()

    # Überprüfe und speichere das Diagramm mit angepasstem Dateinamen
    filename = 'linReg_train.png'
    if os.path.exists(filename):
        filename = 'linReg_test.png'
        print('Prognose mit linearer Regression:')
        print(y_valid_pred)

    plt.show()
    print(y_valid_pred)

#     print('Fehlermetriken für Validierungsdaten:')
#     print('MAE: ', linRegValid_mae)
#     print('MSE: ', linRegValid_mse)
#     print('RMSE: ', linRegValid_rmse)
#     print('MAPE: ', linRegValid_mape)
#     print('R2: ', linRegValid_r2)
#     print()

#     print('Fehlermetriken für Testdaten:')
#     print('MAE: ', linRegTest_mae)
#     print('MSE: ', linRegTest_mse)
#     print('RMSE: ', linRegTest_rmse)
#     print('MAPE: ', linRegTest_mape)
#     print('R2: ', linRegTest_r2)

#     # Daten und Regression für Validierungsdaten plotten
#     plt.figure(figsize=(10, 6))

#     # Plot für Validierungsdaten
#     plt.plot(valid_dates, Y_valid, label='Validierungsdaten (wahre Werte)')
#     plt.plot(valid_dates, y_valid_pred, linestyle='--', label='Validierungsprognose')
#     plt.xlabel('Datum')
#     plt.ylabel('Anzahl')
#     plt.legend()
#     plt.title("Lineare Regression und Prognose für " + name_artikel + " für " + agglo + " (Validierungsdaten)")

#     plt.tight_layout()
#     plt.show()

#     # Daten und Regression für Testdaten plotten
#     plt.figure(figsize=(10, 6))

#     # Plot für Testdaten
#     plt.plot(test_dates, Y_test, label='Testdaten (wahre Werte)')
#     plt.plot(test_dates, y_test_pred, linestyle='--', label='Testprognose')
#     plt.xlabel('Datum')
#     plt.ylabel('Anzahl')
#     plt.legend()
#     plt.title("Lineare Regression und Prognose für " + name_artikel + " für " + agglo + " (Testdaten)")

#     plt.tight_layout()
#     plt.show()
    return y_valid_pred, linRegValid_mae, linRegValid_mse, linRegValid_rmse, linRegValid_mape, linRegValid_r2

In [46]:
# Croston mit Alpha-Schleife auf Trainingsbereich

# Croston Berechnung
def croston(data, alpha):
    # Erstelle eine Kopie des Datenrahmens, um die ursprünglichen Daten nicht zu verändern
    train_data_copy = data.copy()
    
    # Transformiere die Eingabedaten in ein numpy-Array
    d = np.array(train_data_copy['value'])
    
    cols = len(d)  # Länge des historischen Zeitraums

    # level (a), periodicity(p) und forecast (f)
    a, p, f = np.full((3, cols), np.nan)
    q = 1  # Perioden seit der letzten Beobachtung der Nachfrage

    # Initialisierung
    first_occurrence = np.argmax(d > 0)
    a[0] = d[first_occurrence]
    p[0] = 1 + first_occurrence
    f[0] = a[0] / p[0]

    # Erstelle alle t+1 Prognosen
    for t in range(0, cols - 1):  # Angepasster Schleifenbereich
        if d[t] > 0:
            a[t + 1] = alpha * d[t] + (1 - alpha) * a[t]
            p[t + 1] = alpha * q + (1 - alpha) * p[t]
            f[t + 1] = a[t + 1] / p[t + 1]
            q = 1
        else:
            a[t + 1] = a[t]
            p[t + 1] = p[t]
            f[t + 1] = f[t]
            q += 1

    df = pd.DataFrame.from_dict({"Demand": d, "Forecast": f, "Period": p, "Level": a, "Error": d - f})
    
    return df

# Bestes Alpha iterativ suchen
def find_optimal_alpha(data, fehler):
    if fehler == 'MAE':
        best_alpha = None
        best_mae = float('inf')

        # Durchlaufen aller Alpha-Werte im Bereich von 0 bis 1 in 0,01-Schritten
        for alpha in np.arange(0, 1, 0.01):
            # Prognose mit dem aktuellen Alpha-Wert durchführen
            croston_result = croston(data, alpha)

            # Berechnung des MAE für die Prognose
            mae = mean_absolute_error(croston_result['Demand'], croston_result['Forecast'])

            # Überprüfen, ob der aktuelle MAE besser ist als der bisher beste
            if mae < best_mae:
                best_mae = mae
                best_alpha = alpha

        return best_alpha

    elif fehler == 'MSE':
        best_alpha = None
        best_mse = float('inf')

        # Durchlaufen aller Alpha-Werte im Bereich von 0 bis 1 in 0,01-Schritten
        for alpha in np.arange(0, 1, 0.01):
            # Prognose mit dem aktuellen Alpha-Wert durchführen
            croston_result = croston(data, alpha)

            # Berechnung des MAE für die Prognose
            mse = mean_squared_error(croston_result['Demand'], croston_result['Forecast'])

            # Überprüfen, ob der aktuelle MAE besser ist als der bisher beste
            if mse < best_mse:
                best_mse = mse
                best_alpha = alpha

        return best_alpha
    
    elif fehler == 'MAPE':
        best_alpha = None
        best_mape = float('inf')

        # Durchlaufen aller Alpha-Werte im Bereich von 0 bis 1 in 0,01-Schritten
        for alpha in np.arange(0, 1, 0.01):
            # Prognose mit dem aktuellen Alpha-Wert durchführen
            croston_result = croston(data, alpha)

            # Berechnung des MAE für die Prognose
            mape = mean_absolute_percentage_error(croston_result['Demand'], croston_result['Forecast'])

            # Überprüfen, ob der aktuelle MAE besser ist als der bisher beste
            if mape < best_mape:
                best_mape = mape
                best_alpha = alpha

        return best_alpha
    
    elif fehler == 'R2':
        best_alpha = None
        best_r2 = float('inf')

        # Durchlaufen aller Alpha-Werte im Bereich von 0 bis 1 in 0,01-Schritten
        for alpha in np.arange(0, 1, 0.01):
            # Prognose mit dem aktuellen Alpha-Wert durchführen
            croston_result = croston(data, alpha)

            # Berechnung des MAE für die Prognose
            r2 = r2_score(croston_result['Demand'], croston_result['Forecast'])

            # Überprüfen, ob der aktuelle MAE besser ist als der bisher beste
            if r2 < best_r2:
                best_r2 = r2
                best_alpha = alpha

        return best_alpha

    elif fehler == 'RMSE':
        best_alpha = None
        best_rmse = float('inf')

        # Durchlaufen aller Alpha-Werte im Bereich von 0 bis 1 in 0,01-Schritten
        for alpha in np.arange(0, 1, 0.01):
            # Prognose mit dem aktuellen Alpha-Wert durchführen
            croston_result = croston(data, alpha)

            # Berechnung des RMSE für die Prognose
            rmse = np.sqrt(mean_squared_error(croston_result['Demand'], croston_result['Forecast']))

            # Überprüfen, ob der aktuelle RMSE besser ist als der bisher beste
            if rmse < best_rmse:
                best_rmse = rmse
                best_alpha = alpha

        return best_alpha

# Verknüpfung der Croston-Unterprogramme und Diagramm und Fehlerrechnung
def croston80(train_data, test_data, fehler, name_artikel, agglo): 
    best_alpha = find_optimal_alpha(train_data, fehler) # Bestes Alpha finden
    result_train = croston(train_data, best_alpha) # Ergebnis von croston80 mit dem besten Alpha für Trainingsdaten speichern
#     result_test = croston(test_data, best_alpha) # Ergebnis von croston80 mit dem besten Alpha für Testdaten speichern
    result_test = np.full_like(test_data['value'], result_train['Forecast'].iloc[-1])
    
    CrosTest_mae = mean_absolute_error(test_data['value'], result_test)
    CrosTest_mse = mean_squared_error(test_data['value'], result_test)
    CrosTest_mape = mean_absolute_percentage_error(test_data['value'], result_test)
    CrosTest_r2 = r2_score(test_data['value'], result_test)
    CrosTest_rmse = np.sqrt(mean_squared_error(test_data['value'], result_test))
    # Plotten des Forecastes
    
    # Setze die Parameter gemäß den Vorgaben
    fontsize = 12
    fontsize_legende = 8
    figsize = (8, 6)

    plt.figure(figsize=figsize)

    # Plot Trainingsdaten und Prognose für Trainingsdaten
    plt.plot(train_data['date'], train_data['value'], label='Trainingsdaten', color='blue')
    plt.plot(train_data['date'], result_train['Forecast'], label='Trainingsprognose', color='orange')

    # Plot Testdaten und Prognose für Testdaten
    plt.plot(test_data['date'], test_data['value'], label='Testdaten', color='green')
    plt.plot(test_data['date'], result_test, label='Testprognose', linestyle='--', color='red')

    plt.xlabel('Datum', fontsize=fontsize)
    plt.ylabel('Wert', fontsize=fontsize)
#     plt.title('Prognose nach Croston-Methode für Trainings- und Testdaten für ' + name_artikel + ' und ' + agglo, fontsize=fontsize)
    plt.xticks(rotation=45, fontsize=fontsize)
    plt.yticks(fontsize=fontsize)
    plt.legend(fontsize=fontsize_legende)
    plt.grid(True)

    plt.tight_layout()

    # Exportiere das Diagramm als .png
    plt.savefig('croston_train.png')

    plt.show()
    
    # Output Fehler im Testintervall
    
#     print('MAE Valid: ', CrosTest_mae)
#     print('MSE Valid: ', CrosTest_mse)
#     print('MAPE Valid: ', CrosTest_mape)
#     print('R2 Valid: ', CrosTest_r2)
#     print('RMSE Valid: ', CrosTest_rmse)
    return result_test, CrosTest_mae, CrosTest_mse, CrosTest_mape, CrosTest_r2, CrosTest_rmse, best_alpha


In [47]:
# Croston mit Alpha-Schleife auf Trainingsbereich

# Croston Berechnung
def croston(data, alpha):
    # Erstelle eine Kopie des Datenrahmens, um die ursprünglichen Daten nicht zu verändern
    train_data_copy = data.copy()
    
    # Transformiere die Eingabedaten in ein numpy-Array
    d = np.array(train_data_copy['value'])
    
    cols = len(d)  # Länge des historischen Zeitraums

    # level (a), periodicity(p) und forecast (f)
    a, p, f = np.full((3, cols), np.nan)
    q = 1  # Perioden seit der letzten Beobachtung der Nachfrage

    # Initialisierung
    first_occurrence = np.argmax(d > 0)
    a[0] = d[first_occurrence]
    p[0] = 1 + first_occurrence
    f[0] = a[0] / p[0]

    # Erstelle alle t+1 Prognosen
    for t in range(0, cols - 1):  # Angepasster Schleifenbereich
        if d[t] > 0:
            a[t + 1] = alpha * d[t] + (1 - alpha) * a[t]
            p[t + 1] = alpha * q + (1 - alpha) * p[t]
            f[t + 1] = a[t + 1] / p[t + 1]
            q = 1
        else:
            a[t + 1] = a[t]
            p[t + 1] = p[t]
            f[t + 1] = f[t]
            q += 1

    df = pd.DataFrame.from_dict({"Demand": d, "Forecast": f, "Period": p, "Level": a, "Error": d - f})
    
    return df

def croston100(train_valid_data, test_data, fehler, name_artikel, agglo, best_alpha): 
    result_train = croston(train_valid_data, best_alpha) # Ergebnis von croston80 mit dem besten Alpha für Trainingsdaten speichern
    
    # Für den Testbereich einen konstanten Forecast erstellen
    constant_forecast = np.full_like(test_data['value'], result_train['Forecast'].iloc[-1])
    
    print('Prognose für Croston:')
    print(constant_forecast)
    # Metriken für den konstanten Forecast berechnen
    CrosTest_mae = mean_absolute_error(test_data['value'], constant_forecast)
    CrosTest_mse = mean_squared_error(test_data['value'], constant_forecast)
    CrosTest_mape = mean_absolute_percentage_error(test_data['value'], constant_forecast)
    CrosTest_r2 = r2_score(test_data['value'], constant_forecast)
    CrosTest_rmse = np.sqrt(mean_squared_error(test_data['value'], constant_forecast))

    # Setze die Parameter gemäß den Vorgaben
    fontsize = 12
    fontsize_legende = 8
    figsize = (8, 6)

    plt.figure(figsize=figsize)

    # Plot Trainingsdaten und Prognose für Trainingsdaten
    plt.plot(train_valid_data['date'], train_valid_data['value'], label='Trainingsdaten', color='blue')
    plt.plot(train_valid_data['date'], result_train['Forecast'], label='Trainingsprognose', color='orange')

    # Plot Testdaten und konstanten Forecast für Testdaten
    plt.plot(test_data['date'], test_data['value'], label='Testdaten', color='green')
    plt.plot(test_data['date'], constant_forecast, label='Konstanter Forecast', linestyle='--', color='red')

    plt.xlabel('Datum', fontsize=fontsize)
    plt.ylabel('Wert', fontsize=fontsize)
#     plt.title('Prognose nach Croston-Methode für Trainings- und Testdaten für ' + name_artikel + ' und ' + agglo, fontsize=fontsize)
    plt.xticks(rotation=45, fontsize=fontsize)
    plt.yticks(fontsize=fontsize)
    plt.legend(fontsize=fontsize_legende)
    plt.grid(True)

    plt.tight_layout()

    # Exportiere das Diagramm als .png
    plt.savefig('croston_forecast.png')

    plt.show()
    
    
    return constant_forecast, CrosTest_mae, CrosTest_mse, CrosTest_mape, CrosTest_r2, CrosTest_rmse


In [48]:
# Holt-Winters mit TestFehlern und Diagramm

import matplotlib.pyplot as plt

def HoWi80(train_Data, test_Data, agglo, fehler, name_artikel, marker, freq_forecast):
    
    train_data = train_Data.copy()
    test_data = test_Data.copy()

    if freq_forecast == 'D':
        periods = 365
    elif freq_forecast == 'W':
        periods = 52
    elif freq_forecast == 'M':
        periods = 12
    elif freq_forecast == 'Q':
        periods = 4
    else:
        print("Ungültiger Wert für 'freq_forecast'. Bitte wählen Sie 'D', 'W', 'M' oder 'Q'.")
        return

    # Berechnen der Anzahl der Perioden
    n_season = len(train_data) // periods if len(train_data) % periods == 0 else len(train_data) // periods

    if n_season < 2:
        print("Die Anzahl der Perioden ist zu klein für Holt-Winters.")
        print("Das Programm wird abgebrochen.")
        return

    # Prüfen, ob der DataFrame Nullen enthält
    if marker:
        # Multiplikative Holt-Winters Exponential Smoothing (HWES)
        fitted_model_mul = ExponentialSmoothing(train_data['value'], trend='mul', seasonal='mul', seasonal_periods=periods).fit()
        test_predictions_mul = fitted_model_mul.predict(start=test_data.index[0], end=test_data.index[-1])
        
        
#         # Berechnung der Trainings- und Testfehler für die Multiplikative Prognose
#         mae_train_hw_mul = mean_absolute_error(train_data['value'], fitted_model_mul.fittedvalues)
#         mse_train_hw_mul = mean_squared_error(train_data['value'], fitted_model_mul.fittedvalues)
#         mape_train_hw_mul = mean_absolute_percentage_error(train_data['value'], fitted_model_mul.fittedvalues)
#         r2_train_hw_mul = r2_score(train_data['value'], fitted_model_mul.fittedvalues)
        
        HWMulValid_mae = mean_absolute_error(test_data['value'], test_predictions_mul)
        HWMulValid_mse = mean_squared_error(test_data['value'], test_predictions_mul)
        HWMulValid_mape = mean_absolute_percentage_error(test_data['value'], test_predictions_mul)
        HWMulValid_r2 = r2_score(test_data['value'], test_predictions_mul)
        HWMulValid_rmse = np.sqrt(mean_squared_error(test_data['value'], test_predictions_mul))
        
        # Setze die Parameter gemäß den Vorgaben
        fontsize = 12
        fontsize_legende = 8
        figsize = (8, 6)

        # Plot der Multiplikativen Prognose
        plt.figure(figsize=figsize)
        plt.plot(train_data['date'], train_data['value'], label='Trainingsdaten', color='blue')
        plt.plot(test_data['date'], test_data['value'], label='Testdaten', color='green')
        plt.plot(train_data['date'], fitted_model_mul.fittedvalues, label='Modell Trainingsdaten', color='orange')
        plt.plot(test_data['date'], fitted_model_mul.forecast(len(test_data)), linestyle='--', label='Prognose', color='red')
        plt.xlabel('Datum', fontsize=fontsize)
        plt.ylabel('Wert', fontsize=fontsize)
#         plt.title('Multiplikative Holt-Winters Exponential Smoothing', fontsize=fontsize)
        plt.legend(fontsize=fontsize_legende)
        plt.grid(True)

        plt.tight_layout()

        # Exportiere das Diagramm als .png
        plt.savefig('HWMul_train.png')

        plt.show()

    else:
        print("Der DataFrame für", name_artikel, "enthält Nullen, daher wird keine Zerlegung mit model='mul' durchgeführt.")
        print()

    # Additive Holt-Winters Exponential Smoothing (HWES)
    fitted_model_add = ExponentialSmoothing(train_data['value'], trend='add', seasonal='add', seasonal_periods=periods).fit()
    test_predictions_add = fitted_model_add.predict(start=test_data.index[0], end=test_data.index[-1])

#     # Berechnung der Trainings- und Testfehler für die Additive Prognose
#     mae_train_hw_add = mean_absolute_error(train_data['value'], fitted_model_add.fittedvalues)
#     mse_train_hw_add = mean_squared_error(train_data['value'], fitted_model_add.fittedvalues)
#     mape_train_hw_add = mean_absolute_percentage_error(train_data['value'], fitted_model_add.fittedvalues)
#     r2_train_hw_add = r2_score(train_data['value'], fitted_model_add.fittedvalues)

    HWAddValid_mae = mean_absolute_error(test_data['value'], test_predictions_add)
    HWAddValid_mse = mean_squared_error(test_data['value'], test_predictions_add)
    HWAddValid_mape = mean_absolute_percentage_error(test_data['value'], test_predictions_add)
    HWAddValid_r2 = r2_score(test_data['value'], test_predictions_add)
    HWAddValid_rmse = np.sqrt(mean_squared_error(test_data['value'], test_predictions_add))
    
    # Setze die Parameter gemäß den Vorgaben
    fontsize = 12
    fontsize_legende = 8
    figsize = (8, 6)

    # Plot der Additiven Prognose
    plt.figure(figsize=figsize)
    plt.plot(train_data['date'], train_data['value'], label='Trainingsdaten', color='blue')
    plt.plot(test_data['date'], test_data['value'], label='Testdaten', color='green')
    plt.plot(train_data['date'], fitted_model_add.fittedvalues, label='Modell Trainingsdaten', color='orange')
    plt.plot(test_data['date'], fitted_model_add.forecast(len(test_data)), linestyle='--', label='Prognose', color='red')
    plt.xlabel('Datum', fontsize=fontsize)
    plt.ylabel('Wert', fontsize=fontsize)
#     plt.title('Additive Holt-Winters Exponential Smoothing', fontsize=fontsize)
    plt.legend(fontsize=fontsize_legende)
    plt.grid(True)

    plt.tight_layout()

    # Exportiere das Diagramm als .png
    plt.savefig('HWAdd_Train.png')

    plt.show()
    if marker:
        return (test_predictions_mul, test_predictions_add, HWMulValid_mae, HWMulValid_mse, HWMulValid_mape, HWMulValid_r2, HWMulValid_rmse, 
            HWAddValid_mae, HWAddValid_mse, HWAddValid_mape, HWAddValid_r2, HWAddValid_rmse)
    else:
        return (test_predictions_add, HWAddValid_mae, HWAddValid_mse, HWAddValid_mape, HWAddValid_r2, HWAddValid_rmse)


In [49]:
# Holt-Winters mit TestFehlern und Diagramm

import matplotlib.pyplot as plt

def HoWi100(train_Data, test_Data, agglo, fehler, name_artikel, marker, freq_forecast):
    
    train_data = train_Data.copy()
    test_data = test_Data.copy()

    if freq_forecast == 'D':
        periods = 365
    elif freq_forecast == 'W':
        periods = 52
    elif freq_forecast == 'M':
        periods = 12
    elif freq_forecast == 'Q':
        periods = 4
    else:
        print("Ungültiger Wert für 'freq_forecast'. Bitte wählen Sie 'D', 'W', 'M' oder 'Q'.")
        return

    # Berechnen der Anzahl der Perioden
    n_season = len(train_data) // periods if len(train_data) % periods == 0 else len(train_data) // periods

    if n_season < 2:
        print("Die Anzahl der Perioden ist zu klein für Holt-Winters.")
        print("Das Programm wird abgebrochen.")
        return

    # Prüfen, ob der DataFrame Nullen enthält
    if marker:
        # Multiplikative Holt-Winters Exponential Smoothing (HWES)
        fitted_model_mul = ExponentialSmoothing(train_data['value'], trend='mul', seasonal='mul', seasonal_periods=periods).fit()
        test_predictions_mul = fitted_model_mul.predict(start=test_data.index[0], end=test_data.index[-1])
        
        print('Prognose durch Holt-Winters multiplikativ:')
        print(test_predictions_mul)
        
#         # Berechnung der Trainings- und Testfehler für die Multiplikative Prognose
#         mae_train_hw_mul = mean_absolute_error(train_data['value'], fitted_model_mul.fittedvalues)
#         mse_train_hw_mul = mean_squared_error(train_data['value'], fitted_model_mul.fittedvalues)
#         mape_train_hw_mul = mean_absolute_percentage_error(train_data['value'], fitted_model_mul.fittedvalues)
#         r2_train_hw_mul = r2_score(train_data['value'], fitted_model_mul.fittedvalues)
        
        HWMulTest_mae = mean_absolute_error(test_data['value'], test_predictions_mul)
        HWMulTest_mse = mean_squared_error(test_data['value'], test_predictions_mul)
        HWMulTest_mape = mean_absolute_percentage_error(test_data['value'], test_predictions_mul)
        HWMulTest_r2 = r2_score(test_data['value'], test_predictions_mul)
        HWMulTest_rmse = np.sqrt(mean_squared_error(test_data['value'], test_predictions_mul))
        # Setze die Parameter gemäß den Vorgaben
        fontsize = 12
        fontsize_legende = 8
        figsize = (8, 6)
        # Plot der Multiplikativen Prognose
        plt.figure(figsize=figsize)
        plt.plot(train_data['date'], train_data['value'], label='Trainingsdaten', color='blue')
        plt.plot(test_data['date'], test_data['value'], label='Testdaten', color='green')
        plt.plot(train_data['date'], fitted_model_mul.fittedvalues, label='Modell Trainingsdaten', color='orange')
        plt.plot(test_data['date'], fitted_model_mul.forecast(len(test_data)), linestyle='--', label='Prognose', color='red')
        plt.xlabel('Datum', fontsize=fontsize)
        plt.ylabel('Wert', fontsize=fontsize)
#         plt.title('Multiplikative Holt-Winters Exponential Smoothing', fontsize=fontsize)
        plt.legend(fontsize=fontsize_legende)
        plt.grid(True)

        plt.tight_layout()

        # Exportiere das Diagramm als .png
        plt.savefig('HWMul_forecast.png')

        plt.show()

    else:
        print("Der DataFrame für", name_artikel, "enthält Nullen, daher wird keine Zerlegung mit model='mul' durchgeführt.")
        print()

    # Additive Holt-Winters Exponential Smoothing (HWES)
    fitted_model_add = ExponentialSmoothing(train_data['value'], trend='add', seasonal='add', seasonal_periods=periods).fit()
    test_predictions_add = fitted_model_add.predict(start=test_data.index[0], end=test_data.index[-1])

    print('Prognsoe mit Holt-Winters additiv:')
    print(test_predictions_add)
#     # Berechnung der Trainings- und Testfehler für die Additive Prognose
#     mae_train_hw_add = mean_absolute_error(train_data['value'], fitted_model_add.fittedvalues)
#     mse_train_hw_add = mean_squared_error(train_data['value'], fitted_model_add.fittedvalues)
#     mape_train_hw_add = mean_absolute_percentage_error(train_data['value'], fitted_model_add.fittedvalues)
#     r2_train_hw_add = r2_score(train_data['value'], fitted_model_add.fittedvalues)

    HWAddTest_mae = mean_absolute_error(test_data['value'], test_predictions_add)
    HWAddTest_mse = mean_squared_error(test_data['value'], test_predictions_add)
    HWAddTest_mape = mean_absolute_percentage_error(test_data['value'], test_predictions_add)
    HWAddTest_r2 = r2_score(test_data['value'], test_predictions_add)
    HWAddTest_rmse = np.sqrt(mean_squared_error(test_data['value'], test_predictions_add))
    
    # Setze die Parameter gemäß den Vorgaben
    fontsize = 12
    fontsize_legende = 8
    figsize = (8, 6)

    # Plot der Additiven Prognose
    plt.figure(figsize=figsize)
    plt.plot(train_data['date'], train_data['value'], label='Trainingsdaten', color='blue')
    plt.plot(test_data['date'], test_data['value'], label='Testdaten', color='green')
    plt.plot(train_data['date'], fitted_model_add.fittedvalues, label='Modell Trainingsdaten', color='orange')
    plt.plot(test_data['date'], fitted_model_add.forecast(len(test_data)), linestyle='--', label='Prognose', color='red')
    plt.xlabel('Datum', fontsize=fontsize)
    plt.ylabel('Wert', fontsize=fontsize)
#     plt.title('Additive Holt-Winters Exponential Smoothing', fontsize=fontsize)
    plt.legend(fontsize=fontsize_legende)
    plt.grid(True)

    plt.tight_layout()

    # Exportiere das Diagramm als .png
    plt.savefig('HWAdd_forecast.png')

    plt.show()
    if marker:
        return (test_predictions_mul, test_predictions_add, HWMulTest_mae, HWMulTest_mse, HWMulTest_mape, HWMulTest_r2, HWMulTest_rmse, 
            HWAddTest_mae, HWAddTest_mse, HWAddTest_mape, HWAddTest_r2, HWAddTest_rmse)
    else:
        return (test_predictions_add, HWAddTest_mae, HWAddTest_mse, HWAddTest_mape, HWAddTest_r2, HWAddTest_rmse)


In [50]:
# Arima mit Fehlern im Testbereich
# Versuch Arima selbst zu optimieren auf Fehlermetriken

def ARIMA80(train_data, test_data, fehler):
    
    # Arbeitskopie erstellen
    train_copy = train_data.copy()
    test_copy = test_data.copy()

    # Define the p, d and q parameters to take any valve between 0 and 2
    p = d = q = range(0, 3)

    # Generate all different combinations of p, q and q triplets
    pdq = list(itertools.product(p, d, q))


    #     Hyperparameter tuning for SARIMA
    #     Determing p, d, q combinations with MSE scores.

    rows = []

    for param in pdq:
        try:
            mod = statsmodels.tsa.arima.model.ARIMA(train_copy['value'],
                                           order=param,
                                           enforce_stationarity=False,
                                           enforce_invertibility=False)

            results = mod.fit()

            if fehler == 'MAE':
                metric_value = results.mae
            elif fehler == 'MSE':
                metric_value = results.mse
            elif fehler == 'MAPE':
                metric_value = results.mae
            elif fehler == 'RMSE':
                metric_value = results.mse
            elif fehler == 'R2':
                metric_value = results.mae
            else:
                raise ValueError("Ungültige Metrik angegeben.")

            metric_text = 'ARIMA{} - {}: {}'.format(param, fehler, metric_value)
            rows.append([metric_value, metric_text])
        except Exception as e:
            print(f"Fehler beim Modell ARIMA{param}: {e}")
            continue

    metric_results = pd.DataFrame(rows, columns=["metric_value", "Params"])
    metric_results = metric_results.sort_values(by='metric_value', ascending=True)

    # Extrahiere die Parameter aus der obersten Zeile von metric_results
    top_params = metric_results.iloc[0]['Params']
    print(top_params)
    p, d, q = eval(top_params.split(' - ')[0].replace('ARIMA', ''))

    # Erstelle und trainiere das ARIMA-Modell mit den extrahierten Parametern
    model = statsmodels.tsa.arima.model.ARIMA(train_copy['value'],
                                   order=(p, d, q),
                                   enforce_stationarity=False,
                                   enforce_invertibility=False)

    results = model.fit()

    # Modellbewertung mit Testdaten
    fc = results.forecast(steps=len(test_copy))

    # Modellbewertung mit Trainingsdaten
    fc_train = results.forecast(steps=len(train_copy))

    # Berechnen der Fehlermetriken für Testdaten
    ArimaTest_mae = mean_absolute_error(test_copy['value'], fc)
    ArimaTest_mse = mean_squared_error(test_copy['value'], fc)
    ArimaTest_mape = mean_absolute_percentage_error(test_copy['value'], fc)
    ArimaTest_r2 = r2_score(test_copy['value'], fc)
    ArimaTest_rmse =  np.sqrt(mean_squared_error(test_copy['value'], fc))

#     print(f"Mean Absolute Error (MAE) for ARIMA data: {ArimaTest_mae}")
#     print(f"Mean Squared Error (MSE) for ARIMA data: {ArimaTest_mse}")
#     print(f"Mean Absolute Percentage Error (MAPE) for ARIMA data: {ArimaTest_mape}")
#     print(f"R-squared (R^2) for ARIMA data: {ArimaTest_r2}")
#     print(f"RMSE for ARIMA data: {ArimaTest_rmse}")

    # Setze die Parameter gemäß den Vorgaben
    fontsize = 12
    fontsize_legende = 8
    figsize = (8, 6)

    # Plot der Vorhersagen, Trainingsdaten und Testdaten
    plt.figure(figsize=figsize)
    plt.plot(train_copy['date'], train_copy['value'], label='Trainingsdaten', color='blue')
    plt.plot(test_copy['date'], test_copy['value'], label='Testdaten', color='green')
#     plt.plot(train_copy['date'], fc_train, color='blue', label='Vorhersagen (Train)')
    plt.plot(test_copy['date'], fc, color='red', linestyle='--', label='Vorhersagen (Test)')
    plt.xlabel('Datum', fontsize=fontsize)
    plt.ylabel('Wert', fontsize=fontsize)
#     plt.title('ARIMA Vorhersagen vs. Trainings- und Testdaten', fontsize=fontsize)
    plt.legend(fontsize=fontsize_legende)
    plt.grid(True)

    plt.tight_layout()

    # Exportiere das Diagramm als .png
    plt.savefig('arima_train.png')

    plt.show()

    order=(p, d, q)
    return (fc, ArimaTest_mae, ArimaTest_mse, ArimaTest_mape, ArimaTest_r2, ArimaTest_rmse, order)



In [51]:
# Arima mit Fehlern im Testbereich
# Versuch Arima selbst zu optimieren auf Fehlermetriken

def ARIMA100(train_data, test_data, fehler, order):
    
    # Arbeitskopie erstellen
    train_copy = train_data.copy()
    test_copy = test_data.copy()

    
    # Erstelle und trainiere das ARIMA-Modell mit den extrahierten Parametern
    model = statsmodels.tsa.arima.model.ARIMA(train_copy['value'],
                                   order=order,
                                   enforce_stationarity=False,
                                   enforce_invertibility=False)

    results = model.fit()

    # Modellbewertung mit Testdaten
    fc = results.forecast(steps=len(test_copy))

    print('Prognose mit ARIMA:')
    print(fc)
    # Modellbewertung mit Trainingsdaten
    fc_train = results.forecast(steps=len(train_copy))

    # Berechnen der Fehlermetriken für Testdaten
    ArimaTest_mae = mean_absolute_error(test_copy['value'], fc)
    ArimaTest_mse = mean_squared_error(test_copy['value'], fc)
    ArimaTest_mape = mean_absolute_percentage_error(test_copy['value'], fc)
    ArimaTest_r2 = r2_score(test_copy['value'], fc)
    ArimaTest_rmse =  np.sqrt(mean_squared_error(test_copy['value'], fc))

#     print(f"Mean Absolute Error (MAE) for ARIMA data: {ArimaTest_mae}")
#     print(f"Mean Squared Error (MSE) for ARIMA data: {ArimaTest_mse}")
#     print(f"Mean Absolute Percentage Error (MAPE) for ARIMA data: {ArimaTest_mape}")
#     print(f"R-squared (R^2) for ARIMA data: {ArimaTest_r2}")
#     print(f"RMSE for ARIMA data: {ArimaTest_rmse}")

    # Setze die Parameter gemäß den Vorgaben
    fontsize = 12
    fontsize_legende = 8
    figsize = (8, 6)

    # Plot der Vorhersagen, Trainingsdaten und Testdaten
    plt.figure(figsize=figsize)
    plt.plot(train_copy['date'], train_copy['value'], label='Trainingsdaten', color='blue')
    plt.plot(test_copy['date'], test_copy['value'], label='Testdaten', color='green')
#     plt.plot(train_copy['date'], fc_train, color='blue', label='Vorhersagen (Train)')
    plt.plot(test_copy['date'], fc, color='red', linestyle='--', label='Vorhersagen (Test)')
    plt.xlabel('Datum', fontsize=fontsize)
    plt.ylabel('Wert', fontsize=fontsize)
#     plt.title('ARIMA Vorhersagen vs. Trainings- und Testdaten', fontsize=fontsize)
    plt.legend(fontsize=fontsize_legende)
    plt.grid(True)

    plt.tight_layout()

    # Exportiere das Diagramm als .png
    plt.savefig('arima_predictions.png')

    plt.show()

    ARIMA_model = results
    return (fc, ArimaTest_mae, ArimaTest_mse, ArimaTest_mape, ArimaTest_r2, ArimaTest_rmse)



In [52]:
# Sarima mit Fehlern im Testbereich
# Versuch Sarima selbst zu optimieren mit Fehlermetriken


def SARIMA80(train_data, test_data, freq_forecast, fehler):
    
    # Arbeitskopie erstellen
    train_copy = train_data.copy()
    test_copy = test_data.copy()


    if freq_forecast == 'D':
        periods = 365
    elif freq_forecast == 'W':
        periods = 52
    elif freq_forecast == 'M':
        periods = 12
    elif freq_forecast == 'Q':
        periods = 4

    # Define the p, d and q parameters to take any valve between 0 and 2
    p = d = q = range(0, 2)

    # Generate all different combinations of p, q and q triplets
    pdq = list(itertools.product(p, d, q))

    # Generate all different combinations of seasonal p, q and q triplets
    seasonal_pdq = [(x[0], x[1], x[2], periods) for x in list(itertools.product(p, d, q))]

    #     Hyperparameter tuning for SARIMA
    #     Determing p, d, q combinations with MSE scores.

    rows = []

    for param in pdq:
        for param_seasonal in seasonal_pdq:
            try:
                mod = statsmodels.tsa.arima.model.ARIMA(train_copy['value'],
                                               order=param,
                                                seasonal_order=param_seasonal,
                                               enforce_stationarity=False,
                                               enforce_invertibility=False)

                results = mod.fit()

                if fehler == 'MAE':
                    metric_value = results.mae
                elif fehler == 'MSE':
                    metric_value = results.mse
                elif fehler == 'MAPE':
                    metric_value = results.mae
                elif fehler == 'RMSE':
                    metric_value = results.mse
                elif fehler == 'R2':
                    metric_value = results.mae
                else:
                    raise ValueError("Ungültige Metrik angegeben.")

                metric_text = 'SARIMA{}x{}'.format(param,param_seasonal)
                rows.append([metric_value, metric_text])
            except Exception as e:
                print(f"Fehler beim Modell ARIMA{param}: {e}")
                continue

    metric_results = pd.DataFrame(rows, columns=["metric_value", "Params"])
    metric_results = metric_results.sort_values(by='metric_value', ascending=True)

    # Extrahiere die Parameter aus der obersten Zeile von metric_results
    top_params = metric_results.iloc[0]['Params']
    print(top_params)
    # Splitten der Parameter für ARIMA und saisonale Komponente
    arima_params, seasonal_params = top_params.split('x')

    # Extrahiere ARIMA-Parameter
    p, d, q = [int(x) for x in arima_params.replace('SARIMA', '').replace('(', '').replace(')', '').split(',')]

    # Extrahiere saisonale Komponenten-Parameter
    seasonal_p, seasonal_d, seasonal_q, periods = [int(x) for x in seasonal_params.replace('(', '').replace(')', '').split(',')]

    # Erstelle und trainiere das ARIMA-Modell mit den extrahierten Parametern
    model = statsmodels.tsa.arima.model.ARIMA(train_copy['value'],
                                   order=(p, d, q),
                                   seasonal_order=(seasonal_p, seasonal_d, seasonal_q, periods),
                                   enforce_stationarity=False,
                                   enforce_invertibility=False)

    results = model.fit()

    

    # Modellbewertung mit Testdaten
    fc = results.forecast(steps=len(test_copy))

    # Modellbewertung mit Trainingsdaten
    fc_train = results.forecast(steps=len(train_copy))

    # Berechnen der Fehlermetriken für Testdaten
    SarimaTest_mae = mean_absolute_error(test_copy['value'], fc)
    SarimaTest_mse = mean_squared_error(test_copy['value'], fc)
    SarimaTest_mape = mean_absolute_percentage_error(test_copy['value'], fc)
    SarimaTest_r2 = r2_score(test_copy['value'], fc)
    SarimaTest_rmse =  np.sqrt(mean_squared_error(test_copy['value'], fc))

#     print(f"Mean Absolute Error (MAE) for SARIMA data: {SarimaTest_mae}")
#     print(f"Mean Squared Error (MSE) for SARIMA data: {SarimaTest_mse}")
#     print(f"Mean Absolute Percentage Error (MAPE) for SARIMA data: {SarimaTest_mape}")
#     print(f"R-squared (R^2) for SARIMA data: {SarimaTest_r2}")
#     print(f"RMSE for SARIMA data: {SarimaTest_rmse}")

    # Setze die Parameter gemäß den Vorgaben
    fontsize = 12
    fontsize_legende = 8
    figsize = (8, 6)

    # Plot der Vorhersagen, Trainingsdaten und Testdaten
    plt.figure(figsize=figsize)
    plt.plot(train_copy['date'], train_copy['value'], label='Trainingsdaten', color='blue')
    plt.plot(test_copy['date'], test_copy['value'], label='Testdaten', color='green')
    plt.plot(test_copy['date'], fc, color='red', linestyle='--', label='Vorhersagen (Test)')
    plt.xlabel('Datum', fontsize=fontsize)
    plt.ylabel('Wert', fontsize=fontsize)
#     plt.title('SARIMA Vorhersagen vs. Trainings- und Testdaten', fontsize=fontsize)
    plt.legend(fontsize=fontsize_legende)
    plt.grid(True)

    plt.tight_layout()

    # Exportiere das Diagramm als .png
    plt.savefig('sarima_train.png')

    plt.show()

    order=(p, d, q)
    seasonal_order=(seasonal_p, seasonal_d, seasonal_q, periods)
    
    return (fc, SarimaTest_mae, SarimaTest_mse, SarimaTest_mape, SarimaTest_r2, SarimaTest_rmse, order, seasonal_order)



In [53]:
# Sarima mit Fehlern im Testbereich
# Versuch Sarima selbst zu optimieren mit Fehlermetriken


def SARIMA100(train_data, test_data, freq_forecast, fehler, order, seasonal_order):
    
    # Arbeitskopie erstellen
    train_copy = train_data.copy()
    test_copy = test_data.copy()


    if freq_forecast == 'D':
        periods = 365
    elif freq_forecast == 'W':
        periods = 52
    elif freq_forecast == 'M':
        periods = 12
    elif freq_forecast == 'Q':
        periods = 4


    # Erstelle und trainiere das ARIMA-Modell mit den extrahierten Parametern
    model = statsmodels.tsa.arima.model.ARIMA(train_copy['value'],
                                   order=order,
                                   seasonal_order= seasonal_order,
                                   enforce_stationarity=False,
                                   enforce_invertibility=False)

    results = model.fit()

    

    # Modellbewertung mit Testdaten
    fc = results.forecast(steps=len(test_copy))

    print('Prognose Mit SARIMA:')
    print(fc)
    # Modellbewertung mit Trainingsdaten
    fc_train = results.forecast(steps=len(train_copy))

    # Berechnen der Fehlermetriken für Testdaten
    SarimaTest_mae = mean_absolute_error(test_copy['value'], fc)
    SarimaTest_mse = mean_squared_error(test_copy['value'], fc)
    SarimaTest_mape = mean_absolute_percentage_error(test_copy['value'], fc)
    SarimaTest_r2 = r2_score(test_copy['value'], fc)
    SarimaTest_rmse =  np.sqrt(mean_squared_error(test_copy['value'], fc))

#     print(f"Mean Absolute Error (MAE) for SARIMA data: {SarimaTest_mae}")
#     print(f"Mean Squared Error (MSE) for SARIMA data: {SarimaTest_mse}")
#     print(f"Mean Absolute Percentage Error (MAPE) for SARIMA data: {SarimaTest_mape}")
#     print(f"R-squared (R^2) for SARIMA data: {SarimaTest_r2}")
#     print(f"RMSE for SARIMA data: {SarimaTest_rmse}")

    # Setze die Parameter gemäß den Vorgaben
    fontsize = 12
    fontsize_legende = 8
    figsize = (8, 6)

    # Plot der Vorhersagen, Trainingsdaten und Testdaten
    plt.figure(figsize=figsize)
    plt.plot(train_copy['date'], train_copy['value'], label='Trainingsdaten', color='blue')
    plt.plot(test_copy['date'], test_copy['value'], label='Testdaten', color='green')
    plt.plot(test_copy['date'], fc, color='red', linestyle='--', label='Vorhersagen (Test)')
    plt.xlabel('Datum', fontsize=fontsize)
    plt.ylabel('Wert', fontsize=fontsize)
#     plt.title('SARIMA Vorhersagen vs. Trainings- und Testdaten', fontsize=fontsize)
    plt.legend(fontsize=fontsize_legende)
    plt.grid(True)

    plt.tight_layout()

    # Exportiere das Diagramm als .png
    plt.savefig('sarima_predictions.png')

    plt.show()

    return (fc, SarimaTest_mae, SarimaTest_mse, SarimaTest_mape, SarimaTest_r2, SarimaTest_rmse)



In [54]:
# Prophet univariat mit Testfehlern

def prophet80uni(train_data, test_data, freq):
    # Prophet

    # Arbeitskopien erstellen
    train_copy = train_data.copy()
    test_copy = test_data.copy()

    df = pd.DataFrame()

    df['ds'] = pd.to_datetime(train_copy['date'])
    df['y'] = train_copy['value']

    # Prophet-Modell mit Trainingsdaten erstellen und trainieren
    prophet = Prophet()
    prophet.fit(df)

    future = prophet.make_future_dataframe(periods=len(test_copy), freq=freq)
    forecast = prophet.predict(future)
#     forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].head()

    # Setze negative Prognosewerte auf 0
    forecast.loc[forecast['yhat'] < 0, 'yhat'] = 0
    predicted_values_test = forecast.iloc[-len(test_copy):]['yhat']

    # Setze die Parameter gemäß den Vorgaben
    fontsize = 12
    fontsize_legende = 8
    figsize = (8, 6)

    # Daten und Prognose plotten, mit Trainingsdaten, Testdaten, Prognose und tatsächlichen Daten
    fig, ax = plt.subplots(figsize=figsize)
    ax.plot(train_copy['date'], train_copy['value'], label='Trainingsdaten', color='blue')  # Linie für Trainingsdaten hinzufügen
    ax.plot(test_copy['date'], predicted_values_test, label='Prognose', color='red', linestyle='--')
    # ax.scatter(test_copy['date'], test_copy['value'], color='r', label='Testdaten')
    ax.plot(test_copy['date'], test_copy['value'], label='Tatsächliche Daten', color='green')  # Linie für die tatsächlichen Daten hinzufügen
    ax.legend(fontsize=fontsize_legende)
#     ax.set_title("Prophet Prognose für Testdaten", fontsize=fontsize)
    ax.set_xlabel('Datum', fontsize=fontsize)
    ax.set_ylabel('Wert', fontsize=fontsize)

    plt.tight_layout()

    # Setze den Dateinamen für das Speichern des Diagramms
    filename = 'prophet_uni_train.png'
    if os.path.exists(filename):
        filename = 'prophet_uni_forecast.png'
        print('Prognose mit Prophet univariat:')
        print(forecast['yhat'])

    # Plot und speichern des Diagramms
    plt.savefig(filename)
    plt.show()

    plt.show()

    # Fehlermetriken berechnen
    actual_values_test = test_copy['value']
    predicted_values_test = forecast.iloc[-len(test_copy):]['yhat']

    FBPuTest_mae = mean_absolute_error(actual_values_test, predicted_values_test)
    FBPuTest_mse = mean_squared_error(actual_values_test, predicted_values_test)
    FBPuTest_mape = mean_absolute_percentage_error(actual_values_test, predicted_values_test)
    FBPuTest_r2 = r2_score(actual_values_test, predicted_values_test)
    FBPuTest_rmse = np.sqrt(mean_squared_error(actual_values_test, predicted_values_test))
    
    # Fehlermetriken für den Testbereich ausgeben
#     print("Testbereich:")
#     print(f"Mean Absolute Error (MAE): {FBPuTest_mae}")
#     print(f"Mean Squared Error (MSE): {FBPuTest_mse}")
#     print(f"Mean Absolute Percentage Error (MAPE): {FBPuTest_mape}")
#     print(f"R-squared (R^2): {FBPuTest_r2}")
#     print(f"RMSE: {FBPuTest_rmse}")
    
    return (predicted_values_test, FBPuTest_mae, FBPuTest_mse, FBPuTest_mape, FBPuTest_r2, FBPuTest_rmse)


In [55]:
def create_dataset(X, y, time_steps=1):
    Xs, ys = [], []
    
    for i in range(len(X) - time_steps + 1):  # Änderung hier
        v = X.iloc[i:(i + time_steps)].values
        Xs.append(v)        
        ys.append(y.iloc[i + time_steps - 1])  # Änderung hier
    
    return np.array(Xs).astype('float32'), np.array(ys).astype('float32')


def LSTM80 (train_korr, valid_korr, fehler):
    # Arbeitskopien erstellen
    train_copy = train_korr.copy()
    valid_copy = valid_korr.copy()
    
    if 'year' in train_copy.columns and 'month' in train_copy.columns and 'day' in train_copy.columns:
        # Löschen der Spalten 'year', 'month' und 'day' aus train_copy
        train_copy = train_copy.drop(['year', 'month', 'day'], axis=1)

    if 'year' in valid_copy.columns and 'month' in valid_copy.columns and 'day' in valid_copy.columns:
        # Löschen der Spalten 'year', 'month' und 'day' aus valid_copy
        valid_copy = valid_copy.drop(['year', 'month', 'day'], axis=1)


    
    # Trainings- und Validierungsdaten zusammenführen
    df = pd.concat([train_copy, valid_copy], ignore_index=True)
    df.set_index('date', inplace=True)

    # Normalisierung der Daten
    train_max = df.max()
    train_min = df.min()
    train = (df - train_min) / (train_max - train_min)

    # Aufteilen der Daten in Trainings- und Validierungssets
    size_train = len(train_copy)
    df_train = train.iloc[:size_train]
    df_valid = train.iloc[size_train:]

    time_steps = 1

    # Erstellen der Trainings- und Validierungsdatensätze
    X_train, y_train = create_dataset(df_train, df_train['value'], time_steps)
    X_valid, y_valid = create_dataset(df_valid, df_valid['value'], time_steps)

    # LSTM-Modell erstellen
    model_lstm = Sequential(name='lstm_vale3')
    model_lstm.add(LSTM(units=len(df_train.columns), return_sequences=True, input_shape=(X_train.shape[1], X_train.shape[2])))
    model_lstm.add(Dropout(0.2))
    model_lstm.add(LSTM(units=10, return_sequences=True))
    model_lstm.add(Dropout(0.2))
    model_lstm.add(LSTM(units=10, return_sequences=True))
    model_lstm.add(Dropout(0.2))
    model_lstm.add(LSTM(units=10))
    model_lstm.add(Dropout(0.2))
    model_lstm.add(Dense(units=1))
    
    # Auswahl des Verlustfunktion basierend auf der angegebenen Fehlermetrik
    if fehler in ['MSE', 'RMSE']:
        loss_function = 'mean_squared_error'
    elif fehler in ['MAE', 'R2']:
        loss_function = 'mean_absolute_error'
    else:
        loss_function = 'mean_absolute_percentage_error'

    model_lstm.compile(loss=loss_function, optimizer='adam')
    model_lstm.summary()

    # Modelltraining
    history = model_lstm.fit(X_train, y_train, epochs=100, batch_size=30, shuffle=False, validation_data=(X_valid, y_valid), verbose=0)

    y_pred = model_lstm.predict(X_valid)
    y_pred_train = model_lstm.predict(X_train)


    # Rescale the data back to the original scale
    y_valid = y_valid*(train_max[0] - train_min[0]) + train_min[0]
    y_pred = y_pred*(train_max[0] - train_min[0]) + train_min[0]
    y_train = y_train*(train_max[0] - train_min[0]) + train_min[0]
    y_pred_train = y_pred_train*(train_max[0] - train_min[0]) + train_min[0]

    # Setze die Parameter gemäß den Vorgaben
    fontsize = 12
    fontsize_legende = 8
    figsize = (8, 6)

    plt.figure(figsize=figsize)
    plt.plot(df_train.index, y_train, label='Trainingsdaten', color='blue')
    plt.plot(df_valid.index, y_valid, label='Validierungsdaten', color='green')
    plt.plot(df_train.index, y_pred_train, color='orange', label='Vorhersagen (Train)')
    plt.plot(df_valid.index, y_pred, color='red', linestyle='--', label='Vorhersagen (Validierung)')
    # plt.title('LSTM Vorhersagen vs. Trainings- und Validierungsdaten', fontsize=fontsize)
    plt.xlabel('Datum', fontsize=fontsize)
    plt.ylabel('Wert', fontsize=fontsize)
    plt.legend(fontsize=fontsize_legende)
    plt.grid(True)

    plt.tight_layout()

    # Setze den Dateinamen für das Speichern des Diagramms
    filename = 'LSTM_train.png'
    if os.path.exists(filename):
        filename = 'LSTM_forecast.png'
        print('Prognose mit LSTM:')
        print(y_pred)

    # Plot und speichern des Diagramms
    plt.savefig(filename)
    plt.show()

    plt.show()

    # Berechnung der Fehlermetriken für den Testbereich
    LSTMValid_mae = mean_absolute_error(y_valid, y_pred)
    LSTMValid_mse = mean_squared_error(y_valid, y_pred)
    LSTMValid_mape = mean_absolute_percentage_error(y_valid, y_pred)
    LSTMValid_r2 = r2_score(y_valid, y_pred)
    LSTMValid_rmse = np.sqrt(mean_squared_error(y_valid, y_pred))
    
#     print("Validierungsmetriken:")
#     print("MAE (Mean Absolute Error):", LSTMValid_mae)
#     print("MSE (Mean Squared Error):", LSTMValid_mse)
#     print("MAPE (Mean Absolute Percentage Error):", LSTMValid_mape)
#     print("R2 (R-squared):", LSTMValid_r2)
#     print("RMSE: ", LSTMValid_rmse)
    
    return (y_pred, LSTMValid_mae, LSTMValid_mse, LSTMValid_mape, LSTMValid_r2, LSTMValid_rmse)


In [56]:
def RF80 (train_korr, valid_korr, fehler):
    df_train = train_korr.copy()
    df_valid = valid_korr.copy()

    df_train.set_index('date', inplace=True)
    df_valid.set_index('date', inplace=True)

    df_without_target = df_train.columns
    df_without_target = df_without_target.drop('value')

    X_train = df_train[df_without_target]
    y_train = df_train['value']
    X_valid = df_valid[df_without_target]
    y_valid = df_valid['value']

    # RandomForest params dict
    rf_params_one = {}
    if fehler == 'MAE':
        rf_params_one['criterion'] = 'mae'
    elif fehler == 'MAPE':
        rf_params_one['criterion'] = 'mae'
    elif fehler == 'R2':
        rf_params_one['criterion'] = 'mae'
    else:
        rf_params_one['criterion'] = 'mse'
        
    rf_params_one['n_estimators'] = 10
    rf_params_one['max_depth'] = 5
    rf_params_one['max_features'] = None
    rf_params_one['max_leaf_nodes'] = 15
    rf_params_one['min_samples_leaf'] = 1
    rf_params_one['random_state'] = 0
    rf_params_one['n_jobs'] = -1 # run all process

    model_rf_regressor = RandomForestRegressor(**rf_params_one)

    model_rf_regressor.fit(X_train, y_train)

    y_pred_rf_regressor = model_rf_regressor.predict(X_valid)
    y_pred_rf_regressor_train = model_rf_regressor.predict(X_train)

    # Setze die Parameter gemäß den Vorgaben
    fontsize = 12
    fontsize_legende = 8
    figsize = (8, 6)

    plt.figure(figsize=figsize)
    plt.plot(df_train.index, y_train, label='Trainingsdaten', color='blue')
    plt.plot(df_valid.index, y_valid, label='Validierungsdaten', color='green')
    plt.plot(df_train.index, y_pred_rf_regressor_train, color='orange', label='Vorhersagen (Train)')
    plt.plot(df_valid.index, y_pred_rf_regressor, color='red', linestyle='--', label='Vorhersagen (Validierung)')
#     plt.title('RF Vorhersagen vs. Trainings- und Validierungsdaten', fontsize=fontsize)
    plt.xlabel('Datum', fontsize=fontsize)
    plt.ylabel('Wert', fontsize=fontsize)
    plt.legend(fontsize=fontsize_legende)
    plt.grid(True)

    plt.tight_layout()

    # Überprüfe und speichere das Diagramm mit angepasstem Dateinamen
    filename = 'rf_train.png'
    if os.path.exists(filename):
        filename = 'rf_forecast.png'
        print('Prognose mit Random Forest:')
        print(y_pred_rf_regressor)
    plt.savefig(filename)

    plt.show()

    # Berechnung der Metriken für den Validierungsbereich
    RFValid_mae = mean_absolute_error(y_valid, y_pred_rf_regressor)
    RFValid_mse = mean_squared_error(y_valid, y_pred_rf_regressor)
    RFValid_mape = mean_absolute_percentage_error(y_valid, y_pred_rf_regressor)
    RFValid_r2 = r2_score(y_valid, y_pred_rf_regressor)
    RFValid_rmse = np.sqrt(mean_squared_error(y_valid, y_pred_rf_regressor))

    # Ausgabe der Feature Importance
    feature_importance = model_rf_regressor.feature_importances_
    sorted_idx = np.argsort(feature_importance)[::-1]
#     for i, idx in enumerate(sorted_idx):
#         print(f"Feature {i+1}: {X_train.columns[idx]} - Importance: {feature_importance[idx]}")
    
    # Sortiere die Ausgaben nach der Feature-Wichtigkeit
    sorted_importance = sorted(zip(feature_importance, X_train.columns), reverse=True)

    # Extrahiere die sortierten Werte und Features
    sorted_values = [imp[0] for imp in sorted_importance]
    sorted_features = [imp[1] for imp in sorted_importance]

    # Erstelle das Balkendiagramm
    plt.figure(figsize=(8, 6))
    plt.barh(range(len(sorted_features)), sorted_values, align='center')
    plt.yticks(range(len(sorted_features)), sorted_features)
    plt.xlabel('Importance')
#     plt.ylabel('Features')
    plt.title('Feature Importance')
    plt.gca().invert_yaxis()  # Umkehrung der Reihenfolge, damit das wichtigste Feature oben steht

    # Exportiere das Balkendiagramm als Bild
    output_filename = 'feature_importance_train.png'
    
      
    if os.path.exists(output_filename):
        filename = 'feature_importance_test.png'

    plt.savefig(output_filename)
    
    plt.tight_layout()
    plt.savefig(output_filename)
    plt.show()

#     print("MAE (Mean Absolute Error) für den Validierungsbereich:", RFValid_mae)
#     print("MSE (Mean Squared Error) für den Validierungsbereich:", RFValid_mse)
#     print("MAPE (Mean Absolute Percentage Error) für den Validierungsbereich:", RFValid_mape)
#     print("R2 (R-squared) für den Validierungsbereich:", RFValid_r2)
#     print("RMSE: ", RFValid_rmse)
    return (y_pred_rf_regressor, RFValid_mae, RFValid_mse, RFValid_mape, RFValid_r2, RFValid_rmse)


In [57]:
def SVM80 (train_korr, valid_korr):
    df_train = train_korr.copy()
    df_valid = valid_korr.copy()

    df_train.set_index('date', inplace=True)
    df_valid.set_index('date', inplace=True)

    df_without_target = df_train.columns
    df_without_target = df_without_target.drop('value')

    X_train = df_train[df_without_target]
    y_train = df_train['value']
    X_valid = df_valid[df_without_target]
    y_valid = df_valid['value']

    model_svm_regressor = SVR(kernel='rbf', gamma=0.1)

    model_svm_regressor.fit(X_train, y_train)

    y_pred_svm_regressor = model_svm_regressor.predict(X_valid)
    y_pred_svm_regressor_train = model_svm_regressor.predict(X_train)

    # Setze die Parameter gemäß den Vorgaben
    fontsize = 12
    fontsize_legende = 8
    figsize = (10, 6)

    plt.figure(figsize=figsize)
    plt.plot(df_train.index, y_train, label='Trainingsdaten', color='blue')
    plt.plot(df_valid.index, y_valid, label='Validierungsdaten', color='green')
    plt.plot(df_train.index, y_pred_svm_regressor_train, color='orange', label='Vorhersagen (Train)')
    plt.plot(df_valid.index, y_pred_svm_regressor, color='red', linestyle='--', label='Vorhersagen (Validierung)')
#     plt.title('SVM Vorhersagen vs. Trainings- und Validierungsdaten', fontsize=fontsize)
    plt.xlabel('Datum', fontsize=fontsize)
    plt.ylabel('Wert', fontsize=fontsize)
    plt.legend(fontsize=fontsize_legende)
    plt.grid(True)

    plt.tight_layout()

    # Überprüfe und speichere das Diagramm mit angepasstem Dateinamen
    filename = 'svm_train.png'
    if os.path.exists(filename):
        filename = 'svm_forecast.png'
        print('Prognose mit SVM:')
        print(y_pred_svm_regressor)

    plt.savefig(filename)

    plt.show()

    # Berechnung der Metriken für den Validierungsbereich
    SVMValid_mae = mean_absolute_error(y_valid, y_pred_svm_regressor)
    SVMValid_mse = mean_squared_error(y_valid, y_pred_svm_regressor)
    SVMValid_mape = mean_absolute_percentage_error(y_valid, y_pred_svm_regressor)
    SVMValid_r2 = r2_score(y_valid, y_pred_svm_regressor)
    SVMValid_rmse = np.sqrt(mean_squared_error(y_valid, y_pred_svm_regressor))

#     print("MAE (Mean Absolute Error) für den Validierungsbereich:", SVMValid_mae)
#     print("MSE (Mean Squared Error) für den Validierungsbereich:", SVMValid_mse)
#     print("MAPE (Mean Absolute Percentage Error) für den Validierungsbereich:", SVMValid_mape)
#     print("R2 (R-squared) für den Validierungsbereich:", SVMValid_r2)
#     print("RMSE: ", SVMValid_rmse)
    return (y_pred_svm_regressor, SVMValid_mae, SVMValid_mse, SVMValid_mape, SVMValid_r2, SVMValid_rmse)


In [58]:
def prophet80multi(train_korr, valid_korr, selected_countries):
    train_copy = train_korr.copy()
    valid_copy = valid_korr.copy()

    # Umbenennen von Spalten für Prophet
    train_copy.rename(columns={"value": 'y', "date": 'ds'}, inplace=True)
    valid_copy.rename(columns={"value": 'y', "date": 'ds'}, inplace=True)

    # Feiertage 
    years_train = train_copy['ds'].dt.year.unique()
    years_valid = valid_copy['ds'].dt.year.unique()

    # Kombinieren Sie die eindeutigen Jahre aus train und valid
    all_years = np.unique(np.concatenate((years_train, years_valid)))

    # Initialisieren einer leeren Liste für die Feiertagsvariablen
    country_holidays = []

    # Iterieren Sie durch die ausgewählten Länderkürzel und fügen Sie die Feiertagsmodule hinzu
    for country in selected_countries:
        try:
            # Feiertage für das ausgewählte Land und Jahr hinzufügen
            country_holidays.append(getattr(holidays, country)(years=all_years))
        except AttributeError:
            print(f"Feiertage für {country} sind nicht verfügbar oder nicht implementiert.")

    # Kombinieren Sie die Feiertagsvariablen
    all_holidays = sum(country_holidays, holidays.HolidayBase())

    data_holidays = pd.DataFrame.from_dict(all_holidays, orient="index").reset_index()
    data_holidays.columns = ["ds", "holiday"]

    # Prophet-Modell mit Trainingsdaten erstellen und trainieren
    prophet = Prophet(holidays=data_holidays)

    # Prophet-Modell mit zusätzlichen Regressoren erstellen und trainieren
    for col in train_copy.columns:
        if col not in ['ds', 'y']:  # Überspringe die Datumsspalte und die Zielvariable
            prophet.add_regressor(col, standardize=False)

    prophet.fit(train_copy)

    # DataFrame für Vorhersagen für den Validierungsbereich erstellen
    future_valid = pd.DataFrame()

    for col in valid_copy.columns:
        if col not in ['y']:  # Überspringe die Datumsspalte und die Zielvariable
            future_valid[col] = valid_copy[col].values

    forecast_valid = prophet.predict(future_valid)
    forecast_valid[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].head()

    # Setze negative Prognosewerte auf 0
    forecast_valid.loc[forecast_valid['yhat'] < 0, 'yhat'] = 0


    # Fehlermetriken berechnen
    actual_values_valid = valid_copy['y']
    predicted_values_valid = forecast_valid['yhat']

    FBPmValid_mae = mean_absolute_error(actual_values_valid, predicted_values_valid)
    FBPmValid_mse = mean_squared_error(actual_values_valid, predicted_values_valid)
    FBPmValid_mape = mean_absolute_percentage_error(actual_values_valid, predicted_values_valid)
    FBPmValid_r2 = r2_score(actual_values_valid, predicted_values_valid)
    FBPmValid_rmse = np.sqrt(mean_squared_error(actual_values_valid, predicted_values_valid))

    # Fehlermetriken für den Validierungsbereich ausgeben
#     print("Validierungsbereich:")
#     print(f"Mean Absolute Error (MAE): {FBPmValid_mae}")
#     print(f"Mean Squared Error (MSE): {FBPmValid_mse}")
#     print(f"Mean Absolute Percentage Error (MAPE): {FBPmValid_mape}")
#     print(f"R-squared (R^2): {FBPmValid_r2}")
#     print(f"RMSE: {FBPmValid_rmse}")
    
    # Setze die Parameter gemäß den Vorgaben
    fontsize = 12
    fontsize_legende = 8
    figsize = (10, 6)

    fig, ax = plt.subplots(figsize=figsize)
    ax.plot(train_copy['ds'], train_copy['y'], label='Trainingsdaten', color='blue')
    ax.plot(valid_copy['ds'], forecast_valid['yhat'], label='Prognose (Test)', linestyle='--', color='red')
    ax.plot(valid_copy['ds'], valid_copy['y'], label='Tatsächliche Daten', color='green')
    ax.legend(fontsize=fontsize_legende)
#     ax.set_title("Prophet Prognose für Trainings- und Testdaten", fontsize=fontsize)
    ax.set_xlabel('Datum', fontsize=fontsize)
    ax.set_ylabel('Wert', fontsize=fontsize)

    plt.tight_layout()

    # Überprüfe und speichere das Diagramm mit angepasstem Dateinamen
    filename = 'prophet_train_test.png'
    if os.path.exists(filename):
        filename = 'prophet_forecast_test.png'
        print('Prognose mit Prophet multivariat:')
        print(forecast_valid['yhat'])

    plt.savefig(filename)

    plt.show()

    
    return (forecast_valid['yhat'], FBPmValid_mae, FBPmValid_mse, FBPmValid_mape, FBPmValid_r2, FBPmValid_rmse)


In [97]:

# Fehlermetriken für die Validbereiche der Modelle vergleichen
def ergebnis_valid(linRegValid_mae, linRegValid_mse, linRegValid_rmse, linRegValid_mape, linRegValid_r2,
                 CrosValid_mae, CrosValid_mse, CrosValid_rmse, CrosValid_mape, CrosValid_r2,
                 ArimaValid_mae, ArimaValid_mse, ArimaValid_rmse, ArimaValid_mape, ArimaValid_r2,
                 SarimaValid_mae, SarimaValid_mse, SarimaValid_rmse, SarimaValid_mape, SarimaValid_r2,
                 FBPuValid_mae, FBPuValid_mse, FBPuValid_rmse, FBPuValid_mape, FBPuValid_r2,
                 LSTMValid_mae, LSTMValid_mse, LSTMValid_rmse, LSTMValid_mape, LSTMValid_r2,
                 RFValid_mae, RFValid_mse, RFValid_rmse, RFValid_mape, RFValid_r2,
                 SVMValid_mae, SVMValid_mse, SVMValid_rmse, SVMValid_mape, SVMValid_r2,
                 FBPmValid_mae, FBPmValid_mse, FBPmValid_rmse, FBPmValid_mape, FBPmValid_r2,
                 HWAddValid_mae, HWAddValid_mse, HWAddValid_rmse, HWAddValid_mape, HWAddValid_r2,
                 HWMulValid_mae, HWMulValid_mse, HWMulValid_rmse, HWMulValid_mape, HWMulValid_r2, 
                 LSTM_b_Valid_mae, LSTM_b_Valid_mse, LSTM_b_Valid_rmse, LSTM_b_Valid_mape, LSTM_b_Valid_r2):


    # Annahme: Sie haben bereits die Fehlermetriken für jeden Modelltyp im Validierungsbereich berechnet und in Variablen gespeichert

    # Fehlermetriken für das linReg-Modell
    linReg_metrics = [linRegValid_mae, linRegValid_mse, linRegValid_rmse, linRegValid_mape, linRegValid_r2]

    # Fehlermetriken für das Cros-Modell
    Cros_metrics = [CrosValid_mae, CrosValid_mse, CrosValid_rmse, CrosValid_mape, CrosValid_r2]

    # Fehlermetriken für das Arima-Modell
    Arima_metrics = [ArimaValid_mae, ArimaValid_mse, ArimaValid_rmse, ArimaValid_mape, ArimaValid_r2]

    # Fehlermetriken für das FBPu-Modell
    FBPu_metrics = [FBPuValid_mae, FBPuValid_mse, FBPuValid_rmse, FBPuValid_mape, FBPuValid_r2]

    # Fehlermetriken für das Sarima-Modell
    Sarima_metrics = [SarimaValid_mae, SarimaValid_mse, SarimaValid_rmse, SarimaValid_mape, SarimaValid_r2]

    # Fehlermetriken für das LSTM-Modell
    LSTM_metrics = [LSTMValid_mae, LSTMValid_mse, LSTMValid_rmse, LSTMValid_mape, LSTMValid_r2]

    # Fehlermetriken für das RF-Modell
    RF_metrics = [RFValid_mae, RFValid_mse, RFValid_rmse, RFValid_mape, RFValid_r2]

    # Fehlermetriken für das SVM-Modell
    SVM_metrics = [SVMValid_mae, SVMValid_mse, SVMValid_rmse, SVMValid_mape, SVMValid_r2]

    # Fehlermetriken für das FBPm-Modell
    FBPm_metrics = [FBPmValid_mae, FBPmValid_mse, FBPmValid_rmse, FBPmValid_mape, FBPmValid_r2]

    # Fehlermetriken für das LSTM_b-Modell
    LSTM_b_metrics = [LSTM_b_Valid_mae, LSTM_b_Valid_mse, LSTM_b_Valid_rmse, LSTM_b_Valid_mape, LSTM_b_Valid_r2]

    # Fehlermetriken für das HWAdd-Modell (abhängig von 'marker')
    if marker:
        HWAdd_metrics = [HWAddValid_mae, HWAddValid_mse, HWAddValid_rmse, HWAddValid_mape, HWAddValid_r2]
        HWMul_metrics = [HWMulValid_mae, HWMulValid_mse, HWMulValid_rmse, HWMulValid_mape, HWMulValid_r2]
    else:
        HWAdd_metrics = [HWAddValid_mae, HWAddValid_mse, HWAddValid_rmse, HWAddValid_mape, HWAddValid_r2]

    # Liste der Modelle
    if marker: 
        models = ['LinReg', 'Croston', 'ARIMA', 'Prophet-u', 'SARIMA', 'LSTM', 'RF', 'SVR', 'Prophet-m', 'H-W_add', 'H-W_mul', 'LSTM_BFS']
    else:
        models = ['LinReg', 'Croston', 'ARIMA', 'Prophet-u', 'SARIMA', 'LSTM', 'RF', 'SVR', 'Prophet-m', 'H-W_add', 'LSTM_BFS']

    # Erstellen eines DataFrame für den Vergleich der Fehlermetriken im Validierungsbereich
    if marker: 
        comparison_df_valid = pd.DataFrame({
            'Model': models,
            'MAE': [linReg_metrics[0], Cros_metrics[0], Arima_metrics[0], FBPu_metrics[0], Sarima_metrics[0], LSTM_metrics[0], RF_metrics[0], SVM_metrics[0], FBPm_metrics[0], HWAdd_metrics[0], HWMul_metrics[0], LSTM_b_metrics[0]],
            'MSE': [linReg_metrics[1], Cros_metrics[1], Arima_metrics[1], FBPu_metrics[1], Sarima_metrics[1], LSTM_metrics[1], RF_metrics[1], SVM_metrics[1], FBPm_metrics[1], HWAdd_metrics[1], HWMul_metrics[1], LSTM_b_metrics[1]],
            'RMSE': [linReg_metrics[2], Cros_metrics[2], Arima_metrics[2], FBPu_metrics[2], Sarima_metrics[2], LSTM_metrics[2], RF_metrics[2], SVM_metrics[2], FBPm_metrics[2], HWAdd_metrics[2], HWMul_metrics[2], LSTM_b_metrics[2]],
            'MAPE': [linReg_metrics[3], Cros_metrics[3], Arima_metrics[3], FBPu_metrics[3], Sarima_metrics[3], LSTM_metrics[3], RF_metrics[3], SVM_metrics[3], FBPm_metrics[3], HWAdd_metrics[3], HWMul_metrics[3], LSTM_b_metrics[3]],
            'R-squared': [linReg_metrics[4], Cros_metrics[4], Arima_metrics[4], FBPu_metrics[4], Sarima_metrics[4], LSTM_metrics[4], RF_metrics[4], SVM_metrics[4], FBPm_metrics[4], HWAdd_metrics[4], HWMul_metrics[4], LSTM_b_metrics[4]]
        })
    else:
        comparison_df_valid = pd.DataFrame({
            'Model': models,
            'MAE': [linReg_metrics[0], Cros_metrics[0], Arima_metrics[0], FBPu_metrics[0], Sarima_metrics[0], LSTM_metrics[0], RF_metrics[0], SVM_metrics[0], FBPm_metrics[0], HWAdd_metrics[0], LSTM_b_metrics[0]],
            'MSE': [linReg_metrics[1], Cros_metrics[1], Arima_metrics[1], FBPu_metrics[1], Sarima_metrics[1], LSTM_metrics[1], RF_metrics[1], SVM_metrics[1], FBPm_metrics[1], HWAdd_metrics[1], LSTM_b_metrics[1]],
            'RMSE': [linReg_metrics[2], Cros_metrics[2], Arima_metrics[2], FBPu_metrics[2], Sarima_metrics[2], LSTM_metrics[2], RF_metrics[2], SVM_metrics[2], FBPm_metrics[2], HWAdd_metrics[2], LSTM_b_metrics[2]],
            'MAPE': [linReg_metrics[3], Cros_metrics[3], Arima_metrics[3], FBPu_metrics[3], Sarima_metrics[3], LSTM_metrics[3], RF_metrics[3], SVM_metrics[3], FBPm_metrics[3], HWAdd_metrics[3], LSTM_b_metrics[3]],
            'R-squared': [linReg_metrics[4], Cros_metrics[4], Arima_metrics[4], FBPu_metrics[4], Sarima_metrics[4], LSTM_metrics[4], RF_metrics[4], SVM_metrics[4], FBPm_metrics[4], HWAdd_metrics[4], LSTM_b_metrics[4]]
        })
    print(comparison_df_valid)

    # # Überprüfen, ob die Variablen bereits definiert sind, bevor sie gelöscht werden
    # if 'top_mae_models' in globals():
    #     del top_mae_models
    # if 'top_mse_models' in globals():
    #     del top_mse_models
    # if 'top_rmse_models' in globals():
    #     del top_rmse_models
    # if 'top_mape_models' in globals():
    #     del top_mape_models
    # if 'top_r2_models' in globals():
    #     del top_r2_models


    # Für MAE, MSE, RMSE und MAPE die 3 Modelle mit dem geringsten Wert finden
    top_mae_models = comparison_df_valid.nsmallest(3, 'MAE')
    top_mse_models = comparison_df_valid.nsmallest(3, 'MSE')
    top_rmse_models = comparison_df_valid.nsmallest(3, 'RMSE')
    top_mape_models = comparison_df_valid.nsmallest(3, 'MAPE')

    # Für R-squared die 3 Modelle mit dem größten Wert finden
    top_r2_models = comparison_df_valid.nlargest(3, 'R-squared')

    # Erstellen des Säulendiagramms für jede Fehlermetrik
    fig, axs = plt.subplots(3, 2, figsize=(8, 12))

    # MAE
    ax = axs[0, 0]
    bars = ax.bar(comparison_df_valid['Model'], comparison_df_valid['MAE'], color='skyblue')
    ax.set_title('a)', loc='left')
    ax.set_ylabel('MAE')
    ax.set_xlabel('Model')
    ax.tick_params(axis='x', rotation=45)
    # Werte der Balken in 45 Grad über den Balken anzeigen
    y_min, y_max = ax.get_ylim()
    for bar in bars:
        height = bar.get_height()
        text_height = y_min + (y_max - y_min) * 0.05  # 5% der Höhe über der minimalen Y-Achsenhöhe
        ax.text(bar.get_x() + bar.get_width()/2, text_height, f'{height:.2f}', ha='center', va='bottom', rotation=90)

    # MSE
    ax = axs[0, 1]
    bars = ax.bar(comparison_df_valid['Model'], comparison_df_valid['MSE'], color='lightgreen')
    ax.set_title('b)', loc='left')
    ax.set_ylabel('MSE')
    ax.set_xlabel('Model')
    ax.tick_params(axis='x', rotation=45)
    # Werte der Balken in 45 Grad über den Balken anzeigen
    y_min, y_max = ax.get_ylim()
    for bar in bars:
        height = bar.get_height()
        text_height = y_min + (y_max - y_min) * 0.05  # 5% der Höhe über der minimalen Y-Achsenhöhe
        ax.text(bar.get_x() + bar.get_width()/2, text_height, f'{height:.2f}', ha='center', va='bottom', rotation=90)

    # RMSE
    ax = axs[1, 0]
    bars = ax.bar(comparison_df_valid['Model'], comparison_df_valid['RMSE'], color='lightcoral')
    ax.set_title('c)', loc='left')
    ax.set_ylabel('RMSE')
    ax.set_xlabel('Model')
    ax.tick_params(axis='x', rotation=45)
    # Werte der Balken in 45 Grad über den Balken anzeigen
    y_min, y_max = ax.get_ylim()
    for bar in bars:
        height = bar.get_height()
        text_height = y_min + (y_max - y_min) * 0.05  # 5% der Höhe über der minimalen Y-Achsenhöhe
        ax.text(bar.get_x() + bar.get_width()/2, text_height, f'{height:.2f}', ha='center', va='bottom', rotation=90)

    # MAPE
    ax = axs[1, 1]
    bars = ax.bar(comparison_df_valid['Model'], comparison_df_valid['MAPE'], color='salmon')
    ax.set_title('d)', loc='left')
    ax.set_ylabel('MAPE')
    ax.set_xlabel('Model')
    ax.tick_params(axis='x', rotation=45)
    # Werte der Balken in 45 Grad über den Balken anzeigen
    y_min, y_max = ax.get_ylim()
    for bar in bars:
        height = bar.get_height()
        text_height = y_min + (y_max - y_min) * 0.05  # 5% der Höhe über der minimalen Y-Achsenhöhe
        ax.text(bar.get_x() + bar.get_width()/2, text_height, f'{height:.2f}', ha='center', va='bottom', rotation=90)

    # R-squared
    ax = axs[2, 0]
    bars = ax.bar(comparison_df_valid['Model'], comparison_df_valid['R-squared'], color='lightcoral')
    ax.set_title('e)', loc='left')
    ax.set_ylabel('R-squared')
    ax.set_xlabel('Model')
    ax.tick_params(axis='x', rotation=45)
    # Werte der Balken in 45 Grad über den Balken anzeigen
    y_min, y_max = ax.get_ylim()
    for bar in bars:
        height = bar.get_height()
        text_height = y_min + (y_max - y_min) * 0.8  # 5% der Höhe über der minimalen Y-Achsenhöhe
        ax.text(bar.get_x() + bar.get_width()/2, text_height, f'{height:.2f}', ha='center', va='bottom', rotation=90)

#     # Textfeld für die besten Modelle für jede Fehlermetrik
#     for ax, top_models, metric_name in zip(axs.flat, [top_mae_models, top_mse_models, top_rmse_models, top_mape_models, top_r2_models], ['MAE', 'MSE', 'RMSE', 'MAPE', 'R-squared']):
#         best_models_names = top_models['Model'].tolist()
#         best_models_values = top_models[metric_name].tolist()
#         ax.text(0.95, 0.85, f'Best Models for {metric_name}:\n1. {best_models_names[0]}: {best_models_values[0]:.3f}\n2. {best_models_names[1]}: {best_models_values[1]:.3f}\n3. {best_models_names[2]}: {best_models_values[2]:.3f}',
#                 transform=ax.transAxes, fontsize=10, verticalalignment='top', horizontalalignment='right', bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.5))

    plt.tight_layout()
    # Exportieren des Plots als PNG-Datei
    plt.savefig('Fehlerübersicht Train.png', dpi=300, bbox_inches='tight')

    plt.show()
    
    # Printen der Übersicht
    for top_models, metric_name in zip([top_mae_models, top_mse_models, top_rmse_models, top_mape_models, top_r2_models], ['MAE', 'MSE', 'RMSE', 'MAPE', 'R-squared']):
        best_models_names = top_models['Model'].tolist()
        best_models_values = top_models[metric_name].tolist()
        print(f'Best Models for {metric_name}:')
        for i, (model_name, metric_value) in enumerate(zip(best_models_names, best_models_values), start=1):
            print(f'{i}. {model_name}: {metric_value:.3f}')
        print()

    plt.tight_layout()
    plt.show()


In [96]:
# Fehlermetriken für die Testbereiche der Modelle vergleichen
def ergebnis_test(linRegTest_mae, linRegTest_mse, linRegTest_rmse, linRegTest_mape, linRegTest_r2,
                 CrosTest_mae, CrosTest_mse, CrosTest_rmse, CrosTest_mape, CrosTest_r2,
                 ArimaTest_mae, ArimaTest_mse, ArimaTest_rmse, ArimaTest_mape, ArimaTest_r2,
                 SarimaTest_mae, SarimaTest_mse, SarimaTest_rmse, SarimaTest_mape, SarimaTest_r2,
                 FBPuTest_mae, FBPuTest_mse, FBPuTest_rmse, FBPuTest_mape, FBPuTest_r2,
                 LSTMTest_mae, LSTMTest_mse, LSTMTest_rmse, LSTMTest_mape, LSTMTest_r2,
                 RFTest_mae, RFTest_mse, RFTest_rmse, RFTest_mape, RFTest_r2,
                 SVMTest_mae, SVMTest_mse, SVMTest_rmse, SVMTest_mape, SVMTest_r2,
                 FBPmTest_mae, FBPmTest_mse, FBPmTest_rmse, FBPmTest_mape, FBPmTest_r2,
                 HWAddTest_mae, HWAddTest_mse, HWAddTest_rmse, HWAddTest_mape, HWAddTest_r2,
                 HWMulTest_mae, HWMulTest_mse, HWMulTest_rmse, HWMulTest_mape, HWMulTest_r2,
                 LSTM_b_Test_mae, LSTM_b_Test_mse, LSTM_b_Test_rmse, LSTM_b_Test_mape, LSTM_b_Test_r2):
    # Fehlermetriken für das linReg-Modell
    linReg_metrics = [linRegTest_mae, linRegTest_mse, linRegTest_rmse, linRegTest_mape, linRegTest_r2]

    # Fehlermetriken für das Cros-Modell
    Cros_metrics = [CrosTest_mae, CrosTest_mse, CrosTest_rmse, CrosTest_mape, CrosTest_r2]

    # Fehlermetriken für das Arima-Modell
    Arima_metrics = [ArimaTest_mae, ArimaTest_mse, ArimaTest_rmse, ArimaTest_mape, ArimaTest_r2]

    # Fehlermetriken für das FBPu-Modell
    FBPu_metrics = [FBPuTest_mae, FBPuTest_mse, FBPuTest_rmse, FBPuTest_mape, FBPuTest_r2]

    # Fehlermetriken für das Sarima-Modell
    Sarima_metrics = [SarimaTest_mae, SarimaTest_mse, SarimaTest_rmse, SarimaTest_mape, SarimaTest_r2]

    # Fehlermetriken für das LSTM-Modell
    LSTM_metrics = [LSTMTest_mae, LSTMTest_mse, LSTMTest_rmse, LSTMTest_mape, LSTMTest_r2]

    # Fehlermetriken für das RF-Modell
    RF_metrics = [RFTest_mae, RFTest_mse, RFTest_rmse, RFTest_mape, RFTest_r2]

    # Fehlermetriken für das SVM-Modell
    SVM_metrics = [SVMTest_mae, SVMTest_mse, SVMTest_rmse, SVMTest_mape, SVMTest_r2]

    # Fehlermetriken für das FBPm-Modell
    FBPm_metrics = [FBPmTest_mae, FBPmTest_mse, FBPmTest_rmse, FBPmTest_mape, FBPmTest_r2]

    # Fehlermetriken für das LSTM_b-Modell
    LSTM_b_metrics = [LSTM_b_Test_mae, LSTM_b_Test_mse, LSTM_b_Test_rmse, LSTM_b_Test_mape, LSTM_b_Test_r2]

    # Fehlermetriken für das HWAdd-Modell (abhängig von 'marker')
    if marker:
        HWAdd_metrics = [HWAddTest_mae, HWAddTest_mse, HWAddTest_rmse, HWAddTest_mape, HWAddTest_r2]
        HWMul_metrics = [HWMulTest_mae, HWMulTest_mse, HWMulTest_rmse, HWMulTest_mape, HWMulTest_r2]
    else:
        HWAdd_metrics = [HWAddTest_mae, HWAddTest_mse, HWAddTest_rmse, HWAddTest_mape, HWAddTest_r2]

    # Liste der Modelle
    if marker: 
        models = ['LinReg', 'Croston', 'ARIMA', 'Prophet-u', 'SARIMA', 'LSTM', 'RF', 'SVR', 'Prophet-m', 'H-W_add', 'H-W_mul', 'LSTM_BFS']
    else:
        models = ['LinReg', 'Croston', 'ARIMA', 'Prophet-u', 'SARIMA', 'LSTM', 'RF', 'SVR', 'Prophet-m', 'H-W_add', 'LSTM_BFS']

    # Erstellen eines DataFrame für den Vergleich der Fehlermetriken im Validierungsbereich
    if marker: 
        comparison_df = pd.DataFrame({
            'Model': models,
            'MAE': [linReg_metrics[0], Cros_metrics[0], Arima_metrics[0], FBPu_metrics[0], Sarima_metrics[0], LSTM_metrics[0], RF_metrics[0], SVM_metrics[0], FBPm_metrics[0], HWAdd_metrics[0], HWMul_metrics[0], LSTM_b_metrics[0]],
            'MSE': [linReg_metrics[1], Cros_metrics[1], Arima_metrics[1], FBPu_metrics[1], Sarima_metrics[1], LSTM_metrics[1], RF_metrics[1], SVM_metrics[1], FBPm_metrics[1], HWAdd_metrics[1], HWMul_metrics[1], LSTM_b_metrics[1]],
            'RMSE': [linReg_metrics[2], Cros_metrics[2], Arima_metrics[2], FBPu_metrics[2], Sarima_metrics[2], LSTM_metrics[2], RF_metrics[2], SVM_metrics[2], FBPm_metrics[2], HWAdd_metrics[2], HWMul_metrics[2], LSTM_b_metrics[2]],
            'MAPE': [linReg_metrics[3], Cros_metrics[3], Arima_metrics[3], FBPu_metrics[3], Sarima_metrics[3], LSTM_metrics[3], RF_metrics[3], SVM_metrics[3], FBPm_metrics[3], HWAdd_metrics[3], HWMul_metrics[3], LSTM_b_metrics[3]],
            'R-squared': [linReg_metrics[4], Cros_metrics[4], Arima_metrics[4], FBPu_metrics[4], Sarima_metrics[4], LSTM_metrics[4], RF_metrics[4], SVM_metrics[4], FBPm_metrics[4], HWAdd_metrics[4], HWMul_metrics[4], LSTM_b_metrics[4]]
        })
    else:
        comparison_df = pd.DataFrame({
            'Model': models,
            'MAE': [linReg_metrics[0], Cros_metrics[0], Arima_metrics[0], FBPu_metrics[0], Sarima_metrics[0], LSTM_metrics[0], RF_metrics[0], SVM_metrics[0], FBPm_metrics[0], HWAdd_metrics[0], LSTM_b_metrics[0]],
            'MSE': [linReg_metrics[1], Cros_metrics[1], Arima_metrics[1], FBPu_metrics[1], Sarima_metrics[1], LSTM_metrics[1], RF_metrics[1], SVM_metrics[1], FBPm_metrics[1], HWAdd_metrics[1], LSTM_b_metrics[1]],
            'RMSE': [linReg_metrics[2], Cros_metrics[2], Arima_metrics[2], FBPu_metrics[2], Sarima_metrics[2], LSTM_metrics[2], RF_metrics[2], SVM_metrics[2], FBPm_metrics[2], HWAdd_metrics[2], LSTM_b_metrics[2]],
            'MAPE': [linReg_metrics[3], Cros_metrics[3], Arima_metrics[3], FBPu_metrics[3], Sarima_metrics[3], LSTM_metrics[3], RF_metrics[3], SVM_metrics[3], FBPm_metrics[3], HWAdd_metrics[3], LSTM_b_metrics[3]],
            'R-squared': [linReg_metrics[4], Cros_metrics[4], Arima_metrics[4], FBPu_metrics[4], Sarima_metrics[4], LSTM_metrics[4], RF_metrics[4], SVM_metrics[4], FBPm_metrics[4], HWAdd_metrics[4], LSTM_b_metrics[4]]
        })

    print(comparison_df)

    #     # Überprüfen, ob die Variablen bereits definiert sind, bevor sie gelöscht werden
    #     if 'top_mae_models' in globals():
    #         del top_mae_models
    #     if 'top_mse_models' in globals():
    #         del top_mse_models
    #     if 'top_rmse_models' in globals():
    #         del top_rmse_models
    #     if 'top_mape_models' in globals():
    #         del top_mape_models
    #     if 'top_r2_models' in globals():
    #         del top_r2_models

    # Für MAE, MSE, RMSE und MAPE die 3 Modelle mit dem geringsten Wert finden
    top_mae_models = comparison_df.nsmallest(3, 'MAE')
    top_mse_models = comparison_df.nsmallest(3, 'MSE')
    top_rmse_models = comparison_df.nsmallest(3, 'RMSE')
    top_mape_models = comparison_df.nsmallest(3, 'MAPE')

    # Für R-squared die 3 Modelle mit dem größten Wert finden
    top_r2_models = comparison_df.nlargest(3, 'R-squared')

    # Erstellen des Säulendiagramms für jede Fehlermetrik
    fig, axs = plt.subplots(3, 2, figsize=(8, 12))

    # MAE
    ax = axs[0, 0]
    bars = ax.bar(comparison_df['Model'], comparison_df['MAE'], color='skyblue')
    ax.set_title('a)', loc='left')
    ax.set_ylabel('MAE')
    ax.set_xlabel('Model')
    ax.tick_params(axis='x', rotation=45)
    # Werte der Balken in 90 Grad unterhalb der x-Achse anzeigen
    y_min, y_max = ax.get_ylim()
    for bar in bars:
        height = bar.get_height()
        text_height = y_min + (y_max - y_min) * 0.05  # 5% der Höhe über der minimalen Y-Achsenhöhe
        ax.text(bar.get_x() + bar.get_width()/2, text_height, f'{height:.2f}', ha='center', va='bottom', rotation=90)

    # MSE
    ax = axs[0, 1]
    bars = ax.bar(comparison_df['Model'], comparison_df['MSE'], color='lightgreen')
    ax.set_title('b)', loc='left')
    ax.set_ylabel('MSE')
    ax.set_xlabel('Model')
    ax.tick_params(axis='x', rotation=45)
    # Werte der Balken in 90 Grad unterhalb der x-Achse anzeigen
    y_min, y_max = ax.get_ylim()
    for bar in bars:
        height = bar.get_height()
        text_height = y_min + (y_max - y_min) * 0.05  # 5% der Höhe über der minimalen Y-Achsenhöhe
        ax.text(bar.get_x() + bar.get_width()/2, text_height, f'{height:.2f}', ha='center', va='bottom', rotation=90)

    # RMSE
    ax = axs[1, 0]
    bars = ax.bar(comparison_df['Model'], comparison_df['RMSE'], color='lightcoral')
    ax.set_title('c)', loc='left')
    ax.set_ylabel('RMSE')
    ax.set_xlabel('Model')
    ax.tick_params(axis='x', rotation=45)
    # Werte der Balken in 90 Grad unterhalb der x-Achse anzeigen
    y_min, y_max = ax.get_ylim()
    for bar in bars:
        height = bar.get_height()
        text_height = y_min + (y_max - y_min) * 0.05  # 5% der Höhe über der minimalen Y-Achsenhöhe
        ax.text(bar.get_x() + bar.get_width()/2, text_height, f'{height:.2f}', ha='center', va='bottom', rotation=90)

    # MAPE
    ax = axs[1, 1]
    bars = ax.bar(comparison_df['Model'], comparison_df['MAPE'], color='salmon')
    ax.set_title('d)', loc='left')
    ax.set_ylabel('MAPE')
    ax.set_xlabel('Model')
    ax.tick_params(axis='x', rotation=45)
    # Werte der Balken in 90 Grad unterhalb der x-Achse anzeigen
    y_min, y_max = ax.get_ylim()
    for bar in bars:
        height = bar.get_height()
        text_height = y_min + (y_max - y_min) * 0.05  # 5% der Höhe über der minimalen Y-Achsenhöhe
        ax.text(bar.get_x() + bar.get_width()/2, text_height, f'{height:.2f}', ha='center', va='bottom', rotation=90)

    # R-squared
    ax = axs[2, 0]
    bars = ax.bar(comparison_df['Model'], comparison_df['R-squared'], color='lightcoral')
    ax.set_title('e)', loc='left')
    ax.set_ylabel('R-squared')
    ax.set_xlabel('Model')
    ax.tick_params(axis='x', rotation=45)
    # Werte der Balken in 90 Grad unterhalb der x-Achse anzeigen
    y_min, y_max = ax.get_ylim()
    for bar in bars:
        height = bar.get_height()
        text_height = y_min + (y_max - y_min) * 0.8  # 5% der Höhe über der minimalen Y-Achsenhöhe
        ax.text(bar.get_x() + bar.get_width()/2, text_height, f'{height:.2f}', ha='center', va='bottom', rotation=90)

    # Anzeigen der Plots
    plt.tight_layout()
    # Exportieren des Plots als PNG-Datei
    plt.savefig('Fehlerübersicht Test.png', dpi=300, bbox_inches='tight')

    plt.show()

    #     # Textfeld für die besten Modelle für jede Fehlermetrik
    #     for ax, top_models, metric_name in zip(axs.flat, [top_mae_models, top_mse_models, top_rmse_models, top_mape_models, top_r2_models], ['MAE', 'MSE', 'RMSE', 'MAPE', 'R-squared']):
    #         best_models_names = top_models['Model'].tolist()
    #         best_models_values = top_models[metric_name].tolist()
    #         ax.text(0.95, 0.85, f'Best Models for {metric_name}:\n1. {best_models_names[0]}: {best_models_values[0]:.3f}\n2. {best_models_names[1]}: {best_models_values[1]:.3f}\n3. {best_models_names[2]}: {best_models_values[2]:.3f}',
    #                 transform=ax.transAxes, fontsize=10, verticalalignment='top', horizontalalignment='right', bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.5))


    # Anzeigen der Plots
    plt.tight_layout()
    plt.show()

    # Printen der Übersicht
    for top_models, metric_name in zip([top_mae_models, top_mse_models, top_rmse_models, top_mape_models, top_r2_models], ['MAE', 'MSE', 'RMSE', 'MAPE', 'R-squared']):
        best_models_names = top_models['Model'].tolist()
        best_models_values = top_models[metric_name].tolist()
        print(f'Best Models for {metric_name}:')
        for i, (model_name, metric_value) in enumerate(zip(best_models_names, best_models_values), start=1):
            print(f'{i}. {model_name}: {metric_value:.3f}')
        print()
    plt.tight_layout()
    plt.show()



In [108]:
import pandas as pd
import matplotlib.pyplot as plt

# Umwandlung von Cros_4 und LinReg_4 in Pandas-Datenrahmen mit dem Index von Arima_4
Cros_4_df = pd.DataFrame({'Cros_4': Cros_4}, index=Arima_4.index)
LinReg_4_df = pd.DataFrame({'LinReg_4': linReg_4}, index=Arima_4.index)

# Setzen der Schriftgröße
plt.rcParams.update({'font.size': 12})

# Setzen der Größe des Plots
plt.figure(figsize=(8, 6))

# Plot für LinReg_4 (mit Arima-Index)
plt.plot(valid_data['date'], LinReg_4_df, label='LinReg')

# Plot für Cros_4 (mit Arima-Index)
plt.plot(valid_data['date'], Cros_4_df, label='Croston')

# Plot für Arima_4
plt.plot(valid_data['date'], Arima_4, label='ARIMA')

# Plot für Sarima_4
plt.plot(valid_data['date'], Sarima_4, label='SARIMA')

# Plot für valid_data
plt.plot(valid_data['date'], valid_data['value'], label='Validierungsdaten', color='darkgreen', linestyle='--')

# Plot für Trainingsdaten
plt.plot(train_data['date'], train_data['value'], label='Trainingsdaten', color='black', linestyle='--')

# Legende anzeigen außerhalb des Plots (rechts)
plt.legend(loc='center left', bbox_to_anchor=(1, 0.5))

# Achsenbeschriftungen
plt.xlabel('Datum')
plt.ylabel('Absatz [Stück]')

# Gitter anzeigen
plt.grid(True)

# Drehung der x-Achsenbeschriftungen um 45 Grad
plt.xticks(rotation=45)

plt.savefig('LinReg, Cros, Arima, Satima_train_kom.png', bbox_inches='tight')  # Exportieren als PNG
# Plot anzeigen
plt.show()


import pandas as pd
import matplotlib.pyplot as plt

# Umwandlung von Cros_4 und LinReg_4 in Pandas-Datenrahmen mit dem Index von Arima_4
Cros_4_df = pd.DataFrame({'Cros_4': Cros_4}, index=Arima_4.index)
LinReg_4_df = pd.DataFrame({'LinReg_4': linReg_4}, index=Arima_4.index)

# Setzen der Schriftgröße
plt.rcParams.update({'font.size': 12})

# Setzen der Größe des Plots
plt.figure(figsize=(8, 6))

# Plot für LinReg_4 (mit Arima-Index)
plt.plot(valid_data['date'], LinReg_4_df, label='LinReg', linestyle='-.', color='blue')

# Plot für Cros_4 (mit Arima-Index)
plt.plot(valid_data['date'], Cros_4_df, label='Croston', linestyle=':', color='red')

# Plot für Arima_4
plt.plot(valid_data['date'], Arima_4, label='ARIMA', linestyle='-', color='orange')

# Plot für Sarima_4
plt.plot(valid_data['date'], Sarima_4, label='SARIMA', linestyle=(0, (5, 2)), color='purple')

# Plot für valid_data
plt.plot(valid_data['date'], valid_data['value'], label='Testdaten', color='darkgreen', linestyle='--')

# Legende anzeigen außerhalb des Plots (rechts)
plt.legend(loc='center left', bbox_to_anchor=(1, 0.5))

# Achsenbeschriftungen
plt.xlabel('Datum')
plt.ylabel('Absatz [Stück]')

# Gitter anzeigen
plt.grid(True)

# Drehung der x-Achsenbeschriftungen um 45 Grad
plt.xticks(rotation=45)

plt.savefig('LinReg, Cros, Arima, Satima_train_zus.png', bbox_inches='tight')  # Exportieren als PNG

# Plot anzeigen
plt.show()


import pandas as pd
import matplotlib.pyplot as plt

# Setzen der Schriftgröße
plt.rcParams.update({'font.size': 12})

# Setzen der Größe des Plots
plt.figure(figsize=(8, 6))

# Umwandlung von Cros_5 und LinReg_5 in Pandas-Datenrahmen mit dem Index von Arima_5
Cros_5_df = pd.DataFrame({'Cros_5': Cros_5}, index=Arima_5.index)
LinReg_5_df = pd.DataFrame({'LinReg_5': linReg_5}, index=Arima_5.index)

# Plot für LinReg_5 (mit Arima-Index)
plt.plot(test_data['date'], LinReg_5_df, label='LinReg', linestyle='-.', color='blue')

# Plot für Cros_5 (mit Arima-Index)
plt.plot(test_data['date'], Cros_5_df, label='Croston', linestyle=':', color='red')

# Plot für Arima_5
plt.plot(test_data['date'], Arima_5, label='ARIMA', linestyle='-', color='orange')

# Plot für Sarima_5
plt.plot(test_data['date'], Sarima_5, label='SARIMA', linestyle=(0,(5, 2)), color='purple')

# Plot für valid_data
# Filtern der letzten 20% der Trainingsdaten
train_data_last_20_percent = train_valid_data.tail(int(len(train_valid_data) * 0.2))

plt.plot(train_data_last_20_percent['date'], train_data_last_20_percent['value'], label='Trainingsdaten', color='black', linestyle='--')


# Plot Testdaten
plt.plot(test_data['date'], test_data['value'], label='Validierungsdaten', color='darkgreen', linestyle='--')

# Legende anzeigen außerhalb des Plots (rechts)
plt.legend(loc='center left', bbox_to_anchor=(1, 0.5))

# Achsenbeschriftungen
plt.xlabel('Datum')
plt.ylabel('Absatz [Stück]')

# Drehung der x-Achsenbeschriftungen um 45 Grad
plt.xticks(rotation=45)
# Gitter anzeigen
plt.grid(True)


plt.savefig('LinReg, Cros, Arima, Satima_test_kom.png', bbox_inches='tight')  # Exportieren als PNG

# Plot anzeigen
plt.show()


import pandas as pd
import matplotlib.pyplot as plt

# Setzen der Schriftgröße
plt.rcParams.update({'font.size': 12})

# Setzen der Größe des Plots
plt.figure(figsize=(8, 6))

# Umwandlung von Cros_5 und LinReg_5 in Pandas-Datenrahmen mit dem Index von Arima_5
Cros_5_df = pd.DataFrame({'Cros_5': Cros_5}, index=Arima_5.index)
LinReg_5_df = pd.DataFrame({'LinReg_5': linReg_5}, index=Arima_5.index)

# Plot für LinReg_5 (mit Arima-Index)
plt.plot(test_data['date'], LinReg_5_df, label='LinReg')

# Plot für Cros_5 (mit Arima-Index)
plt.plot(test_data['date'], Cros_5_df, label='Croston')

# Plot für Arima_5
plt.plot(test_data['date'], Arima_5, label='ARIMA')

# Plot für Sarima_5
plt.plot(test_data['date'], Sarima_5, label='SARIMA')

# Plot für valid_data
# plt.plot(train_valid_data['date'], train_valid_data['value'], label='Validierungsdaten', color='black', linestyle='--')
plt.plot(test_data['date'], test_data['value'], label='Trainingsdaten', color='darkgreen', linestyle='--')

# Legende anzeigen außerhalb des Plots (rechts)
plt.legend(loc='center left', bbox_to_anchor=(1, 0.5))

# Achsenbeschriftungen
plt.xlabel('Datum')
plt.ylabel('Absatz [Stück]')



# Gitter anzeigen
plt.grid(True)
plt.savefig('LinReg, Cros, Arima, Satima_test_zus.png', bbox_inches='tight')  # Exportieren als PNG

# Drehung der x-Achsenbeschriftungen um 45 Grad
plt.xticks(rotation=45)

# Plot anzeigen
plt.show()


In [107]:
import pandas as pd
import matplotlib.pyplot as plt

# Setzen der Schriftgröße
plt.rcParams.update({'font.size': 12})

# Setzen der Größe des Plots
plt.figure(figsize=(8, 6))

# Plot für LinReg_4 (mit Arima-Index)
plt.plot(valid_data['date'], FBPu_4, label='Prophet-u')

# Plot für HWAdd_4
plt.plot(valid_data['date'], HWAdd_4, label='H-W_add')

# Plot für HWMul_4
plt.plot(valid_data['date'], HWMul_4, label='H-W_mul')

# Plot für valid_data
plt.plot(valid_data['date'], valid_data['value'], label='Validierungsdaten', color='darkgreen', linestyle='--')

# Plot für Trainingsdaten
plt.plot(train_data['date'], train_data['value'], label='Trainingsdaten', color='black', linestyle='--')

# Legende anzeigen außerhalb des Plots (rechts)
plt.legend(loc='center left', bbox_to_anchor=(1, 0.5))

# Achsenbeschriftungen
plt.xlabel('Datum')
plt.ylabel('Absatz [Stück]')

# Gitter anzeigen
plt.grid(True)

# Drehung der x-Achsenbeschriftungen um 45 Grad
plt.xticks(rotation=45)
plt.savefig('FB, HW_train_kom.png', bbox_inches='tight')  # Exportieren als PNG
# Plot anzeigen
plt.show()


import pandas as pd
import matplotlib.pyplot as plt


# Setzen der Schriftgröße
plt.rcParams.update({'font.size': 12})

# Setzen der Größe des Plots
plt.figure(figsize=(8, 6))

# Plot für LinReg_4 (mit Arima-Index)
plt.plot(valid_data['date'], FBPu_4, label='Prophet-u', linestyle='-.', color='blue')

# Plot für HWAdd_4
plt.plot(valid_data['date'], HWAdd_4, label='H-W_add', linestyle=':', color='red')

# Plot für HWMul_4
plt.plot(valid_data['date'], HWMul_4, label='H-W_mul', linestyle='-', color='orange')

# Plot für valid_data
plt.plot(valid_data['date'], valid_data['value'], label='Testdaten', color='darkgreen', linestyle='--')

# Legende anzeigen außerhalb des Plots (rechts)
plt.legend(loc='center left', bbox_to_anchor=(1, 0.5))

# Achsenbeschriftungen
plt.xlabel('Datum')
plt.ylabel('Absatz [Stück]')

# Gitter anzeigen
plt.grid(True)

# Drehung der x-Achsenbeschriftungen um 45 Grad
plt.xticks(rotation=45)

plt.savefig('FB, HW_train_zus.png', bbox_inches='tight')  # Exportieren als PNG

# Plot anzeigen
plt.show()


import pandas as pd
import matplotlib.pyplot as plt

# Setzen der Schriftgröße
plt.rcParams.update({'font.size': 12})

# Setzen der Größe des Plots
plt.figure(figsize=(8, 6))

# Plot für LinReg_5 (mit Arima-Index)
plt.plot(test_data['date'], FBPu_5, label='Prophet-u', linestyle='-.', color='blue')

# Plot für HWAdd_5
plt.plot(test_data['date'], HWAdd_5, label='H-W_add', linestyle=':', color='red')

# Plot für HWMul_5
plt.plot(test_data['date'], HWMul_5, label='H-W_mul', linestyle='-', color='orange')

# Plot für valid_data
# Filtern der letzten 20% der Trainingsdaten
train_data_last_20_percent = train_valid_data.tail(int(len(train_valid_data) * 0.2))

plt.plot(train_data_last_20_percent['date'], train_data_last_20_percent['value'], label='Trainingsdaten', color='black', linestyle='--')


# Plot Testdaten
plt.plot(test_data['date'], test_data['value'], label='Validierungsdaten', color='darkgreen', linestyle='--')

# Legende anzeigen außerhalb des Plots (rechts)
plt.legend(loc='center left', bbox_to_anchor=(1, 0.5))

# Achsenbeschriftungen
plt.xlabel('Datum')
plt.ylabel('Absatz [Stück]')

# Drehung der x-Achsenbeschriftungen um 45 Grad
plt.xticks(rotation=45)
# Gitter anzeigen
plt.grid(True)

plt.savefig('FB, HW_test_kom.png', bbox_inches='tight')  # Exportieren als PNG

# Plot anzeigen
plt.show()


import pandas as pd
import matplotlib.pyplot as plt

# Setzen der Schriftgröße
plt.rcParams.update({'font.size': 12})

# Setzen der Größe des Plots
plt.figure(figsize=(8, 6))


# Plot für LinReg_5 (mit Arima-Index)
plt.plot(test_data['date'], FBPu_5, label='Prophet-u')

# Plot für HWAdd_5
plt.plot(test_data['date'], HWAdd_5, label='H-W_add')

# Plot für HWMul_5
plt.plot(test_data['date'], HWMul_5, label='H-W_mul')

# Plot für valid_data
# plt.plot(train_valid_data['date'], train_valid_data['value'], label='Validierungsdaten', color='black', linestyle='--')
plt.plot(test_data['date'], test_data['value'], label='Trainingsdaten', color='darkgreen', linestyle='--')

# Legende anzeigen außerhalb des Plots (rechts)
plt.legend(loc='center left', bbox_to_anchor=(1, 0.5))

# Achsenbeschriftungen
plt.xlabel('Datum')
plt.ylabel('Absatz [Stück]')



# Gitter anzeigen
plt.grid(True)

# Drehung der x-Achsenbeschriftungen um 45 Grad
plt.xticks(rotation=45)

plt.savefig('FB, HW_test_zus.png', bbox_inches='tight')  # Exportieren als PNG

# Plot anzeigen
plt.show()


In [106]:
import pandas as pd
import matplotlib.pyplot as plt

# Setzen der Schriftgröße
plt.rcParams.update({'font.size': 12})

# Setzen der Größe des Plots
plt.figure(figsize=(8, 6))

# Plot für LSTM_4
plt.plot(valid_korr['date'],LSTM_4, label='LSTM')

# Plot für RF_4
plt.plot(valid_korr['date'],RF_4, label='RF')

# Plot für SVM_4
plt.plot(valid_korr['date'],SVM_4, label='SVR')

# Plot für FBPm_4
plt.plot(valid_korr['date'],FBPm_4, label='Prophet-m')

# Plot für LSTM_b_4
plt.plot(valid_korr['date'],LSTM_b_4, label='LSTM_BFS')

# Plot für valid_data
plt.plot(valid_korr['date'], valid_korr['value'], label='Validierungsdaten', color='darkgreen', linestyle='--')

# Plot für Trainingsdaten
plt.plot(train_korr['date'], train_korr['value'], label='Trainingsdaten', color='black', linestyle='--')

# Legende anzeigen außerhalb des Plots (rechts)
plt.legend(loc='center left', bbox_to_anchor=(1, 0.5))

# Achsenbeschriftungen
plt.xlabel('Datum')
plt.ylabel('Absatz [Stück]')

# Gitter anzeigen
plt.grid(True)

# Drehung der x-Achsenbeschriftungen um 45 Grad
plt.xticks(rotation=45)
plt.savefig('multi_train_kom.png', bbox_inches='tight')  # Exportieren als PNG

# Plot anzeigen
plt.show()


import pandas as pd
import matplotlib.pyplot as plt


# Setzen der Schriftgröße
plt.rcParams.update({'font.size': 12})

# Setzen der Größe des Plots
plt.figure(figsize=(8, 6))

# Plot für LSTM_4
plt.plot(valid_korr['date'],LSTM_4, label='LSTM', linestyle='-.', color='blue')

# Plot für RF_4
plt.plot(valid_korr['date'],RF_4, label='RF', linestyle=':', color='red')

# Plot für SVM_4
plt.plot(valid_korr['date'],SVM_4, label='SVR', linestyle='-', color='orange')

# Plot für FBPm_4
plt.plot(valid_korr['date'],FBPm_4, label='Prophet-m', linestyle=(0,(10,2)), color='purple')

# Plot für LSTM_b_4
plt.plot(valid_korr['date'],LSTM_b_4, label='LSTM_BFS', linestyle=(0,(2,5)), color='brown')

# Plot für valid_data
plt.plot(valid_korr['date'], valid_korr['value'], label='Testdaten', color='darkgreen', linestyle='--')

# Legende anzeigen außerhalb des Plots (rechts)
plt.legend(loc='center left', bbox_to_anchor=(1, 0.5))

# Achsenbeschriftungen
plt.xlabel('Datum')
plt.ylabel('Absatz [Stück]')

# Gitter anzeigen
plt.grid(True)

# Drehung der x-Achsenbeschriftungen um 45 Grad
plt.xticks(rotation=45)
plt.savefig('multi_train_zus.png', bbox_inches='tight')  # Exportieren als PNG

# Plot anzeigen
plt.show()


import pandas as pd
import matplotlib.pyplot as plt

# Setzen der Schriftgröße
plt.rcParams.update({'font.size': 12})

# Setzen der Größe des Plots
plt.figure(figsize=(8, 6))

# Plot für LSTM_4
plt.plot(test_korr['date'],LSTM_5, label='LSTM', linestyle='-.', color='blue')

# Plot für RF_4
plt.plot(test_korr['date'],RF_5, label='RF', linestyle=':', color='red')

# Plot für SVM_4
plt.plot(test_korr['date'],SVM_5, label='SVR', linestyle='-', color='orange')

# Plot für FBPm_4
plt.plot(test_korr['date'],FBPm_5, label='Prophet-m', linestyle=(0,(10,2)), color='purple')

# Plot für LSTM_b_4
plt.plot(test_korr['date'],LSTM_b_5, label='LSTM_BFS', linestyle=(0,(2,5)), color='brown')

# Plot für valid_data
# Filtern der letzten 20% der Trainingsdaten
train_korr_last_20_percent = train_valid_korr.tail(int(len(train_valid_korr) * 0.2))

plt.plot(train_korr_last_20_percent['date'], train_korr_last_20_percent['value'], label='Trainingsdaten', color='black', linestyle='--')


# Plot Testdaten
plt.plot(test_korr['date'], test_korr['value'], label='Validierungsdaten', color='darkgreen', linestyle='--')

# Legende anzeigen außerhalb des Plots (rechts)
plt.legend(loc='center left', bbox_to_anchor=(1, 0.5))

# Achsenbeschriftungen
plt.xlabel('Datum')
plt.ylabel('Absatz [Stück]')

# Drehung der x-Achsenbeschriftungen um 45 Grad
plt.xticks(rotation=45)
# Gitter anzeigen
plt.grid(True)
plt.savefig('multi_test_kom.png', bbox_inches='tight')  # Exportieren als PNG

# Plot anzeigen
plt.show()


import pandas as pd
import matplotlib.pyplot as plt

# Setzen der Schriftgröße
plt.rcParams.update({'font.size': 12})

# Setzen der Größe des Plots
plt.figure(figsize=(8, 6))

# Plot für LSTM_4
plt.plot(test_korr['date'],LSTM_5, label='LSTM')

# Plot für RF_4
plt.plot(test_korr['date'],RF_5, label='RF')

# Plot für SVM_4
plt.plot(test_korr['date'],SVM_5, label='SVR')

# Plot für FBPm_4
plt.plot(test_korr['date'],FBPm_5, label='Prophet-m')

# Plot für LSTM_b_4
plt.plot(test_korr['date'],LSTM_b_5, label='LSTM_BFS')

# Plot für valid_data
# plt.plot(train_valid_data['date'], train_valid_data['value'], label='Validierungsdaten', color='black', linestyle='--')
plt.plot(test_korr['date'], test_korr['value'], label='Trainingsdaten', color='darkgreen', linestyle='--')

# Legende anzeigen außerhalb des Plots (rechts)
plt.legend(loc='center left', bbox_to_anchor=(1, 0.5))

# Achsenbeschriftungen
plt.xlabel('Datum')
plt.ylabel('Absatz [Stück]')



# Gitter anzeigen
plt.grid(True)

# Drehung der x-Achsenbeschriftungen um 45 Grad
plt.xticks(rotation=45)

plt.savefig('multi_test_zus.png', bbox_inches='tight')  # Exportieren als PNG

# Plot anzeigen
plt.show()
